# Rapid-DSM

# Import Python Libraries

In [ ]:
!sudo apt-get install jq

In [ ]:
import os
import folium
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import json
from IPython.display import Image

# This is where pip is installing modules so we need to add it to the system path.
os.environ['PATH'] += ":/root/.local/bin"

!pip install geopandas
import geopandas as gpd

!pip install rasterio
import rasterio

!pip install planet --user #v1.4.6




# Authenticate Goolge Features

#### Google CoLab Authentication

In [ ]:
from google.colab import auth
auth.authenticate_user()

#### Mount Google Drive

We will store figures here

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Google Earth Engine Cli

In [ ]:
!earthengine authenticate

#### Google Earth Engine for Python

In [ ]:
# Import the Earth Engine library.
import ee

# Trigger the authentication flow.
ee.Authenticate()
ee.Initialize()

# Default Configuration

#### Goolge Cloud Storage

In [ ]:
os.environ['GCP_PROJECT_ID'] = project_id = 'rapid-dsm'
planet_gcp_directory = 'planet_scope'
os.environ['PL_GCP_SERVICE_ACCOUNT'] = planet_service_account = "<Your Google Cloud Platform Serivce Account>"
os.environ['PL_GCP_CREDS_JSON'] = google_creds_json = "creds.json"

## Google Drive Save Location
figures_save_location = "<Your Google Drive Save Location>"

## Google Earth Engien Data
os.environ['GEE_PROJECT_FOLDER'] = gee_project_folder = "<Your Google Earth Engine Project Folder>"
os.environ['GEE_IMAGECOLLECTION'] = imageCollection = os.path.join(gee_project_folder, "planetscope")
os.environ['GEE_IMAGECOLLECTION_AOI'] = gee_imagecollection_aoi = os.path.join(gee_project_folder, "aoi")


In [ ]:
from google.cloud import storage
storage_client = storage.Client(project=project_id)

## AOI: Walnut Creek Subwatershed

In [ ]:
# Data from Google Earth Engine 
# HUC12: USGS Watershed Boundary Dataset of Subwatersheds
subwatershed = ee.FeatureCollection("USGS/WBD/2017/HUC12")
aoi = (subwatershed.filter(
  ee.Filter.And(
    ee.Filter.eq("huc12", "030202011101"),
    ee.Filter.eq("states", "NC"),
    ee.Filter.eq("name", "Walnut Creek")
    )
  ))

url = aoi.getDownloadUrl(filetype="GEOJSON")
df_aoi = gpd.read_file(url)
# df_aoi.plot()
df_envelope = gpd.GeoDataFrame(geometry=gpd.GeoSeries(df_aoi['geometry'].envelope))
df_envelope.plot()
print(df_envelope.area)
aoi = df_envelope.to_file("aoi.json", driver='GeoJSON')
#make shapefile for GEE later
df_envelope.to_file("aoi")
with open("aoi.json", "r") as f:
  aoi_json = json.loads(f.read())
  print(aoi_json['features'][0]['geometry'])
  os.environ['AOI_STRING'] = AOI_STRING = str(aoi_json['features'][0]['geometry'])

## Random AOI that happens to capture the two AOIS

In [ ]:
# original 
os.environ['AOI_STRING'] = AOI_STRING = '{"type": "Polygon","coordinates": [[[-78.79113513335422,35.73453757221374],	[-78.654824101583,35.73453757221374],[-78.654824101583,35.78778630839446],[-78.79113513335422,35.78778630839446],[-78.79113513335422,35.73453757221374]]]}'
!touch aoi.json
!echo '{"type": "Polygon","coordinates": [[[-78.79113513335422,35.73453757221374],	[-78.654824101583,35.73453757221374],[-78.654824101583,35.78778630839446],[-78.79113513335422,35.78778630839446],[-78.79113513335422,35.73453757221374]]]}' > aoi.json

In [ ]:
# Set from https://planetlabs.github.io/planet-client-python/cli/reference.html
os.environ['PL_API_KEY'] = planet_api_key = "<Your PLANET API KEY>"
# Max cloud cover set at %10
os.environ['PL_MAX_CLOUD_COVER'] = "0.05"
# Look angle setting the off-Nadir filer settings
os.environ['PL_MIN_LOOK'] = "-2"
os.environ['PL_MAX_LOOK'] = "2"
# Note: Sun Elevation - set in future form the images found in the inital search
os.environ['PL_BUCKET'] = planet_bucket = 'gs://%s' % planet_gcp_directory
os.environ['PL_TEMP_DATA'] = planet_tmp_data = 'planet'
os.environ['PL_ASSET_TYPE'] = planet_asset_type = 'analytic_sr'
os.environ['PL_ITEM_TYPE'] = planet_item_type = 'PSScene4Band'
os.environ['PL_ITEM_IDS'] = pl_item_ids = ''
os.environ['PL_BUNDLE'] = 'analytic_sr'
os.environ['PL_TOOLS_FILE'] = planet_order_tool_file = "tools.json"
os.environ['PL_CLOUD_CONFIG'] = planet_cloudconfig = 'pl_cloudconfig.json'
os.environ['AOI_JSON'] = aoi_json = 'aoi.json'
os.environ['AOI_SHAPEFILE_DIR'] = aoi_shapefile_dir = 'aoi/'


In [ ]:
# Create a temporary directory to store results from Planet request
!mkdir $PL_TEMP_DATA

### User Input

In [ ]:
#@title Select a date range for your study area
#@markdown Forms support many types of fields.
start_date = '2018-01-01'  #@param {type: "date"}
end_date = '2020-09-01'  #@param {type: "date"}
#@markdown ---

os.environ['PL_DATE_START'] = start_date
os.environ['PL_DATE_END'] = end_date
os.environ['PL_ORDER_NAME'] = order_name = "rapid-dsm_{}_{}".format(start_date,end_date)

# Get Planet Data

### Generate Planet Delivery JSON

Planet's Orders API allows for direct delivery to cloud storage options such as Google Could Platform. To enable this service a bucket with Read and Write permissions must be generated for a specified service account. A new access key must be created and the returned json encoded as base64 for use with Planet's api. The below code will generate your bucket and set acl parameters and generate and encode google credentials for use with Planet's API. 

In [ ]:
!gsutil ls -p $GCP_PROJECT_ID | grep $PL_BUCKET

In [ ]:
# Create delivery file so we can send download directly to Google Storage
# API DOCS: https://developers.planet.com/docs/orders/delivery/

# Add permissions to for Planet to read and write data to google cloud storage bucket.
!gsutil acl ch -u $PL_GCP_SERVICE_ACCOUNT:WR $PL_BUCKET

# https://cloud.google.com/docs/authentication/getting-started#command-line
# Download credential json file for your google service account
!gcloud iam service-accounts keys create $PL_GCP_CREDS_JSON --iam-account $PL_GCP_SERVICE_ACCOUNT

# Encode cred.json as base64 so you can pass it as a planet order argument
planet_cloud_storage_creds = !cat $PL_GCP_CREDS_JSON | base64 | tr -d '\n'

delivery = {
    "google_cloud_storage": {
        "bucket": planet_gcp_directory,
        "credentials": planet_cloud_storage_creds[0],
        "path_prefix": planet_gcp_directory
        }
    }

with open(planet_cloudconfig, "w+") as m:
  m.write(json.dumps(delivery))


### Search Planet Data

In [ ]:
# Use planet cli to search for data
!planet -k $PL_API_KEY data search --limit 500 --item-type $PL_ITEM_TYPE --asset-type $PL_ASSET_TYPE --date acquired gt $PL_DATE_START --date acquired lt $PL_DATE_END --range cloud_cover lte $PL_MAX_CLOUD_COVER --string-in quality_category "standard" --range view_angle lt $PL_MAX_LOOK --range view_angle gt $PL_MIN_LOOK --geom aoi.json > ps_search.json


In [ ]:
aoi = gpd.read_file('aoi.json')
search_results = gpd.read_file('ps_search.json')

In [ ]:
search_results.describe()[['cloud_cover','gsd', 'sun_azimuth', 'sun_elevation', 'view_angle']]

In [ ]:
start_image = search_results['acquired'].min()
end_image = search_results['acquired'].max()
print("Images from %s to %s" % (start_image, end_image))


In [ ]:
from matplotlib.dates import DateFormatter

w_test = search_results.sort_values("acquired").set_index(pd.DatetimeIndex(search_results['acquired']))
w_test_count = w_test.groupby(pd.Grouper(freq="M")).count()
w_test_count['count'] = w_test_count['id']
w_test_count.head()
# fig, ax = plt.subplots(figsize=(9, 6))
fig, ax = plt.subplots(figsize=(9, 6))
# ax.xaxis_date()
g = w_test_count.plot(y='count', kind="bar", ax=ax)
g.axvline(x=12, linewidth=2, color='r',linestyle="--", label="2019-01-22")
# date_form = DateFormatter("%m-%d-%Y")
# g.xaxis.set_major_formatter(date_form)
plt.xticks(rotation=45)
plt.suptitle('PlanetScope Image Date Range', y=1.01, fontsize=16)
plt.title("Midpoint date 2019-01-22", fontsize=14) 
plt.savefig(os.path.join(figures_save_location,"PlanetScope Image Date Range"),bbox_inches='tight')

# 2019-01-22T14:59:44.500000


In [ ]:
coverage = search_results.plot(column='acquired', alpha=0.5, cmap="tab10")
aoi.plot(color='white', edgecolor='black', ax=coverage)

### Create Planet Data Order

Get a list of Planet item ids to order for delivery.

In [ ]:
item_ids = search_results["id"].tolist()

# pl_order['products'][0]['item_ids'] = item_ids

# Set this environmental variable to be used in Planet API order request.
os.environ['PL_ITEM_IDS'] = pl_item_ids = ','.join(map(str, item_ids))

In [ ]:
#https://developers.planet.com/docs/orders/ordering/
# !planet -k $PL_API_KEY orders create --name $PL_ORDER_NAME  --bundle $PL_BUNDLE  --clip $AOI_JSON --item-type $PL_ITEM_TYPE --id $PL_ITEM_IDS --cloudconfig $PL_CLOUD_CONFIG
#Removed Clip tool for CGA account
!planet -k $PL_API_KEY orders create --name $PL_ORDER_NAME  --bundle $PL_BUNDLE --item-type $PL_ITEM_TYPE --id $PL_ITEM_IDS --cloudconfig $PL_CLOUD_CONFIG

In [ ]:
pl_orders = !planet -k $PL_API_KEY orders list | jq -r ".orders[0].id" 
# Get last Planet order ID to search for downloaded data
os.environ['PL_ORDER_ID'] =order_id = pl_orders.pop()

### Wait for Planet Order to complete

In [ ]:
!planet -k $PL_API_KEY orders get $PL_ORDER_ID | jq -r ".last_message"

# Upload Planet Data to Google Earth Engine

Before you can upload data to planet **you first need to create an ImageCollection in Google Earth Engine** to store the Images. The following script will load the data into the ImageCollection.

> Google Earth Engine (GEE) doesn't provide a great API for this functionality yet. Ideally, you could just have GEE digest data directly from a Google Cloud Storage Bucket.

https://github.com/samapriya/geeup provides another option for doing this, but I found it buggy and prone to breakages becasue of how it's implenented using a shadow browser to script through GEE UI.



## Create GEE ImageCollection if needed

In [ ]:
!earthengine --no-use_cloud_api create folder $GEE_PROJECT_FOLDER





In [ ]:
!earthengine --no-use_cloud_api create collection $GEE_IMAGECOLLECTION
!earthengine --no-use_cloud_api asset set --time_start $PL_DATE_START --time_end $PL_DATE_END $GEE_IMAGECOLLECTION

#Add search parameters to imagecollection metadata
# !earthengine --no-use_cloud_api asset set -p '(string)name=42' users/username/asset_id


## Upload Area of interest to GEE

In [ ]:
# It wants to read the file form cloud storage, so copy it to the project folder
!gsutil -m cp -r $AOI_SHAPEFILE_DIR $PL_BUCKET/"planet_scope"/$PL_ORDER_ID/
!earthengine --no-use_cloud_api upload table --asset_id=$GEE_PROJECT_FOLDER/aoi $PL_BUCKET/"planet_scope"/$PL_ORDER_ID/aoi/aoi.shp

### Copy manifest files from Planet Scope Bucket

In [ ]:
#copy planet image metadata to local directory to use during Google Earth Engine upload.
!gsutil -m cp $PL_BUCKET/"planet_scope"/$PL_ORDER_ID/$PL_ITEM_TYPE/*.json $PL_TEMP_DATA

Create Manifest file to send to to Google Earth Engine for data Ingest form Google Cloud Storage Bucket.

In [ ]:
from datetime import datetime
dt_format = "%Y-%m-%dT%H:%M:%S"

# Get the start and end date of the image range to set manually on the ImageCollection
# start_dt = datetime.strptime(start_image, dt_format)
# end_dt = datetime.strptime(end_image, dt_format)
# print(start_dt, end_dt)


"""
Builds a manifest.json file for each image getting sent to GEE. The manifest file includes all metadata 
provided by PlanetScope allowing it to be accessable via GEE.
"""
for dirpath, dirnames, filenames in os.walk(planet_tmp_data):
  for filename in filenames:
    if filename.endswith(".json"):
      with open(os.path.join(dirpath, filename)) as json_file:
        
        manifest = {
          "name": imageCollection,
          "tilesets": [
            {
              "sources": [
                {
                  "uris": []
                }
              ]
            }
          ],
          "start_time": {
            "seconds": None 
          }
        }
        data = json.load(json_file)
        image_name = filename.split("_metadata.json")[0] + "_3B_AnalyticMS_SR.tif"
        manifest["name"] = os.path.join(imageCollection, data['id'])
        
        url = os.path.join(planet_bucket, planet_gcp_directory, order_id,planet_item_type, image_name)
        row = data["properties"]
        # row["system:time_start"] = datetime.strptime(row["acquired"],"%Y-%m-%dT%H:%M:%S.%fz").timestamp()
        row["number_of_bands"] = 4
        manifest['start_time']['seconds'] = int(datetime.strptime(row['acquired'],"%Y-%m-%dT%H:%M:%S.%fz").timestamp())
        manifest["tilesets"][0]["id"] = "ps"
        del row["ground_control"]
        manifest["properties"] = row
        manifest["tilesets"][0]["sources"][0]['uris'].append(url)
        # manifest["tilesets"][0]["sources"].append(data_ps4bsr)
        manifest_file_name = "manifest_" + data["id"] + '.json'
        print(manifest_file_name)
        with open(manifest_file_name, 'w+') as m:
          m.write(json.dumps(manifest))
        !earthengine upload image --manifest {manifest_file_name}
        

# Change Detection

## Land Cover Land Change Value Mappings

In [ ]:
# OSM key value pairs to incldue
# white_list = [
#       #  'highway_residential', 'highway_primary','highway_motorway', 'highway_trunk',
#        'landuse_grass', 'landuse_meadow',
#        'natural_water', 'natural_wood', "natural_grassland",
#        'surface_sand', 'surface_grass','surface_dirt',
#        'parking_surface',
#        'building_residential', 'building_retail','building_public', 'building_house']

# Maps OSM key values to localized schema
land_covers = {
    'highway_residential': "road",
    'highway_motorway': "road",
    'highway_trunk': "road",
    'highway_primary': "road",

    'highway_secondary': "road",
    'highway_tertiary': "road",
    'highway_unclassified': "road",
    'highway_motorway_link': "road",
    'highway_trunk_link': "road",
    'highway_primary_link': "road",
    'highway_secondary_link': "road",
    'highway_tertiary_link': "road",
    'parking_surface': "developed",
    # 'landuse_residential': "residential",
    # 'landuse_construction': "barren",
    # 'landuse_retail': "retail",
    # 'surface_sand': "barren",
    # 'surface_ground': "barren",
    # 'surface_dirt': "barren",
    'surface_grass': "grass",
    'landuse_grass': "grass", 
    'landuse_meadow': "grass",
    "natural_grassland": "grass",
    'natural_water': "water",
    # 'natural_wetland': "wetland",
    'natural_wood': "forest",
    'building_house': "building",
    'building_residential': "building",
    'building_retail': "building",
    'building_public': "building"
    # 'highway_construction': "highway_construction"
  }

query_keys = tuple(set([k.split('_')[0] for k in land_covers.keys()]))
query_values = tuple(set([k.split('_')[1] for k in land_covers.keys()]))
print(query_keys)
# Map of human readable landclasses to coded values
land_classes = {
    "road": 0, # Developed
    "building": 1, #Developed
    "barren": 2,
    "forest": 3,
    "grass": 4, #Herbaceous
    "water": 5,
    "developed": 6
  }

#colors from https://www.mrlc.gov/data/legends/national-land-cover-database-2011-nlcd2011-legend
landcover_color_palette = [
    'E29E8C', # Class 0 - road/highways         NLCD class color 22
    'B50000', # Class 1 - Building dark red     NLCD class color 24
    'D2CDC0', # Class 2 - barren                NLCD class color 31
    '38814E', # Class 3 - Forest                NLCD class color 42
    '85C77E', # Class 4 - Grass                 NLCD class color 41
    '5475A8', # Class 5 -  water                 NLCD class color 11
    'E8D1D1' # Class 5 -  developed             NLCD class color 21
]

#### Priority Change Config

In [ ]:
# Scale the change importance 

# 0 = No Change
# 1 = Probable Noise
# 2 = Low 
# 3 = Medium
# 4 = High

change_priority = {
    "No Change": 0,
    "road to building": 7,
    "road to barren": 4,
    "road to water": 0,
    "road to grass": 1,
    "road to forest": 1,
    "road to developed": 4,
    "building to road": 1,
    "building to barren": 7,
    "building to water": 0,
    "building to grass": 3,
    "building to forest": 3,
    "building to developed": 5,
    "barren to road": 3,
    "barren to building": 7,
    "barren to water": 0,
    "barren to grass": 2,
    "barren to forest": 2,
    "barren to developed": 5,
    "water to road": 0,
    "water to building": 0,
    "water to barren": 0,
    "water to grass": 0,
    "water to forest": 0,
    "water to developed": 0,
    "grass to road": 3,
    "grass to building": 7,
    "grass to barren": 3,
    "grass to water":0,
    "grass to forest": 3,
    "grass to developed": 5,
    "forest to road":	3,
    "forest to building": 7,
    "forest to barren": 7,
    "forest to water":	0,
    "forest to grass":	3,
    "forest to developed":	7,
    "developed to road":	3,
    "developed to building": 7,
    "developed to barren": 7,
    "developed to water":	0,
    "developed to grass":	3,
    "developed to forest":	3
}



## Functions

In [ ]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, eeImageObject, visParams, name):
  mapID = ee.Image(eeImageObject).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles = mapID['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

def createNDVI(image):
  ndvi = image.normalizedDifference(['b4', 'b3']).select(["nd"],["ndvi"])
  return ndvi

def createNDWI(image):
  ndwi = image.normalizedDifference(['b1', 'b4']).select(["nd"], ["ndwi"])
  return ndwi

def createNDCI(image):
  ndci = image.normalizedDifference(['b4', 'b2']).select(["nd"], ["ndci"])
  return ndci

def createBSI(image):
  #https://openprairie.sdstate.edu/cgi/viewcontent.cgi?article=4165&context=etd
  bsi = image.expression(
    '((RED + GREEN) - (RED + BLUE))/ ((NIR + GREEN) + (RED + BLUE)) * 100 + 100' , {
      'NIR': image.select('b4'),
      'RED': image.select('b3'),
      'BLUE': image.select('b1'),
      'GREEN': image.select('b2')
    }).select(['b3'],['bsi'])
  return bsi

def createDSBI(image):
  #Lingjia GuLingjia 2018
  dsbi = image.expression(
    '0.5 * (BLUE - RED) +0.5*(BLUE - GREEN)' , {
      'NIR': image.select('b4'),
      'RED': image.select('b3'),
      'BLUE': image.select('b1'),
      'GREEN': image.select('b2')
    }).select(['constant'],['dsbi'])
  return dsbi

def createBSI_NDVI_index(image_bsi, image_ndvi):
  image = image_bsi.addBands(image_ndvi)
  ndbsvi = image.normalizedDifference(['bsi', 'ndvi']).select(["nd"], ["ndbsiv"])
  return ndbsvi

def createFeatureImportanceBarChart(classifier, label=""):
  fig, ax = plt.subplots(figsize=(9, 6))
  classifier_dict = classifier.explain()
  variable_importance = ee.Feature(None, ee.Dictionary(classifier_dict).get('importance'))
  
  print("Variable Importance")
  props = variable_importance.getInfo()["properties"]
  data = [{"feature":v, "value":props[v]} for v in props]
  df_variable_importance = pd.DataFrame(data)
  plt.title('%s Feature Importance' % label.capitalize(), fontsize=14) 
  sns.barplot(x="value", y="feature", data=df_variable_importance.sort_values("value", ascending=False),
            label="Feature Importance", color="b")
  plt.savefig(os.path.join(figures_save_location,label + "_FeatImportance"))
  return df_variable_importance.sort_values("value", ascending=False)['feature']


def generateFromToExpression(landclasses):
  """
  Generates thematic change expression and labeling dictionary
  @param landclasses land_class dictionary 
  """
  output = ""
  from_to_labels = {
      "No Change": 0
  }
  for form_value_key in land_classes:
    from_value = land_classes[form_value_key]
    for to_value_key in land_classes:
      to_value = land_classes[to_value_key]
      if from_value != to_value:
        change_class_value = str(from_value) + str(to_value)
        change_class_key = "%s to %s" % (form_value_key, to_value_key)
        from_to_labels[change_class_key] = change_class_value
        base_text = "(b('classification') == {0} && b('classification_1') == {1}) ? {2} :".format(from_value, to_value, change_class_value)
        output = output + base_text
  output = output + " 0"
  return({"labels": from_to_labels, "expression": output})

result = generateFromToExpression(land_classes)
thematic_change_expression = result['expression']
print(result['labels'])
print(thematic_change_expression)
tmp_change_class_data = [{"FromTo":v, "ClassID":result["labels"][v]} for v in result["labels"]]
df_change_classes = pd.DataFrame(tmp_change_class_data)
df_change_classes["priority"] = df_change_classes["FromTo"].map(change_priority)

def generateThematicChangeImage(from_image, to_image, expression):
  # Add classified bands to same image
  temp = from_image.addBands(to_image)
  change_image = temp.expression(expression)
  return change_image

#This helper function returns a list of new band names.
def getNewBandNames(prefix, bandNames):
  seq = ee.List.sequence(1, len(bandNames))
  z = [prefix + str(ee.Number(b).int().getInfo()) for b in seq.getInfo()]
  return z
  # return seq.map(lambda b: ee.String("%s%d") % (prefix, ee.Number(b).int()))
  # return seq.map(lambda b: ee.String(prefix).cat(str(ee.Number(b).int())))

#This function accepts mean centered imagery, a scale and
#a region in which to perform the analysis.  It returns the
#Principal Components (PC) in the region as a new image.
def getPrincipalComponents(centered, scale, region):
  #Collapse the bands of the image into a 1D array per pixel.
  arrays = centered.toArray();
    
  #Compute the covariance of the bands within the region.
  covar = arrays.reduceRegion(
      reducer = ee.Reducer.centeredCovariance(), 
      geometry= region,
      scale= scale, 
      maxPixels=1e9, 
      bestEffort=True, 
      tileScale=16)
   
  #Get the 'array' covariance result and cast to an array.
  #This represents the band-to-band covariance within the region.
  covarArray = ee.Array(covar.get('array'))
    
  #Perform an eigen analysis and slice apart the values and vectors.
  eigens = covarArray.eigen().getInfo()
  # print("eigens", eigens)
  #This is a P-length vector of Eigenvalues.
  eigenValues = ee.Array(eigens).slice(1, 0, 1);
  # print("eigenValues", eigenValues)
  #This is a PxP matrix with eigenvectors in rows.
  eigenVectors = ee.Array(eigens).slice(1, 1);
  # print("eigenVectors", eigenVectors)
  #Convert the array image to 2D arrays for matrix computations.
  arrayImage = arrays.toArray(1)
  # print("arrayImage", arrayImage)


  #Left multiply the image array by the matrix of eigenvectors.
  principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

  #Turn the square roots of the Eigenvalues into a P-band image.
  sdImage = ee.Image(eigenValues.sqrt())
  sdImage = sdImage.arrayProject([0])
  bandNames = centered.bandNames().getInfo()
  sdImage = sdImage.arrayFlatten([getNewBandNames('sd', bandNames)])

  #Turn the PCs into a P-band image, normalized by SD.
  pc = principalComponents.arrayProject([0])#Throw out an an unneeded dimension, [[]] -> [].
  pc = pc.arrayFlatten([getNewBandNames('pc',bandNames)])#Make the one band array image a multi-band image, [] -> image
  pc = pc.divide(sdImage)#Normalize the PCs by their SDs.
  return pc

def createConfusionMatixFigure(testAccuracy, label=""):
  fig, ax = plt.subplots(figsize=(9, 6))
  df_confusion_test = pd.DataFrame(testAccuracy.getInfo(), index=list(land_classes.keys()), columns=list(land_classes.keys()))
  plt.title('%s Confusion Matrix' % label.capitalize(), fontsize=14) 
  sns.heatmap(df_confusion_test, annot=True, fmt="d", linewidths=.5, ax=ax, cmap="Blues")
  plt.savefig(os.path.join(figures_save_location,label + "_ConfMatrix"))

### Function: Export exportEarthEngineImage(image, desc, imageName, region, saveLocation="GoogleDrive")

In [ ]:
import time

def exportEarthEngineImage(image, desc, imageName, region,scale=3, saveLocation="GoogleDrive"):
  if (saveLocation == "CloudStorage"):
    imageTask = ee.batch.Export.image.toCloudStorage(
      image=image,
      description=desc,
      fileNamePrefix=imageName,
      bucket="classification-results",
      scale=scale,
      fileFormat='GeoTIFF',
      skipEmptyTiles=True,
      maxPixels=1e12,
      # maxZoom=16,
      crs='EPSG:3857',
      formatOptions= {
      "cloudOptimized": True
      },
      #dimensions=256,
      # fileDimensions=256,
      #shardSize=256
      )
  elif (saveLocation == "GoogleDrive"):
    imageTask = ee.batch.Export.image.toDrive(
    image=image,
    folder=figures_save_location,
    description='Image Export %s' % imageName,
    fileNamePrefix=imageName,
    scale=3,
    fileFormat= 'GeoTIFF',
    formatOptions= {"cloudOptimized": True},
    region=region
    )
  elif (saveLocation == "Asset"):
    imageTask = ee.batch.Export.image.toAsset(
      assetId= imageName,
      image=image,
      description='Image Export',
      fileNamePrefix=imageName,
      scale=3,
      fileFormat='GeoTIFF',
      region=region.toGeoJSON()['coordinates']
    )
  else:
    print("Unknown Save Location, must be either 'GoogleDrive','CloudStorage', or 'Asset")
    exit

  imageTask.start()
  while imageTask.active():
    print('Polling for task (id: {}).'.format(imageTask.id))
    time.sleep(3)


def exportToDrive(image, imageLabel, resolution=30):
  """
  param: image : ee.Image
  param: imageLabel: string
  param: resolution : export resolution (default = 30)
  """
  ee.batch.Export.image.toDrive(
    image=image,
    folder=figures_save_location,
    description=imageLabel,
    fileNamePrefix=imageLabel,
    scale=resolution,
    fileFormat= 'GeoTIFF',
    region=aoi.geometry(),
    formatOptions= {"cloudOptimized": True}).start()

In [ ]:
# aoi = ee.Geometry.Polygon([
# 							[-78.79113513335422,35.73453757221374],	
# 							[-78.654824101583,35.73453757221374],
# 							[-78.654824101583,35.78778630839446],
# 							[-78.79113513335422,35.78778630839446],
# 							[-78.79113513335422,35.73453757221374]
# 						])

# aoi = ee.Feature(g)

aoi_bbox = ee.FeatureCollection(gee_imagecollection_aoi)

subwatershed = ee.FeatureCollection("USGS/WBD/2017/HUC12")
aoi = (subwatershed.filter(
  ee.Filter.And(
    ee.Filter.eq("huc12", "030202011101"),
    ee.Filter.eq("states", "NC"),
    ee.Filter.eq("name", "Walnut Creek")
    )
  ))
# print(aoi.first())
coords = aoi.geometry().centroid().getInfo()['coordinates']
center_map = [coords[1], coords[0]]


#For paper figures
centenial = ee.Geometry.Polygon([
	        		[-78.68571281433105, 35.76256750220092],
          		[-78.66464138031006, 35.76256750220092],
            	[-78.66464138031006, 35.778723954134776],
            	[-78.68571281433105, 35.778723954134776],
            	[-78.68571281433105, 35.76256750220092 ]                        
])



cary_park = ee.Geometry.Polygon([
	        		[
              -78.78489017486572,
              35.77962917544667
            ],
            [
              -78.77480506896973,
              35.77962917544667
            ],
            [
              -78.77480506896973,
              35.78788014067825
            ],
            [
              -78.78489017486572,
              35.78788014067825
            ],
            [
              -78.78489017486572,
              35.77962917544667
            ]                       
])


## Download OpenStreetMap Data

In [ ]:
# Query Google Big Query to generate training data for image classification.

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
#old tag values removing roads to see if I can fix the max 
#AND tags.value in ("water", "grass","meadow","grassland", "forest", "residential","secondary","tertiary","unclassified", "primary", "motorway", "trunk","motorway_link","trunk_link","primary_link","secondary_link","tertiary_link", "quarry", "farmland", "sand", "ground", "dirt", "wood", "retail","public", "construction", "surface", "house") 

df_landcover_samples = pd.io.gbq.read_gbq('''
  SELECT feature_type,osm_timestamp, tags.key, tags.value, ST_ASGEOJSON(geometry) as geom
  FROM `bigquery-public-data.geo_openstreetmap.planet_features` AS feats,
  UNNEST(feats.all_tags) AS tags
  WHERE tags.key in ("building", "landuse", "highway", "water", "natural", "surface", "parking") 
      AND tags.value in ("water", "grass","meadow","grassland", "forest", "residential","secondary", "primary", "motorway", "trunk","trunk_link","primary_link","secondary_link","tertiary_link", "quarry", "farmland", "sand", "ground", "dirt", "wood", "retail","public", "construction", "surface", "house") 
      AND ST_INTERSECTS(feats.geometry, ST_GEOGFROMGEOJSON('{"type": "Polygon","coordinates": [[[ -78.782195642777026, 35.692691564447983 ],[ -78.531885144268472, 35.692691564447983 ],[ -78.531885144268472, 35.800084957942794 ],[ -78.782195642777026, 35.800084957942794 ],[ -78.782195642777026, 35.692691564447983 ]] ]}'))
  ''', project_id=project_id)

 #project_id is defined in config
# Create dataframe column that represents the OSM key value pair
df_landcover_samples["keyvalue"] = df_landcover_samples["key"] + "_" + df_landcover_samples["value"]

# Map land cover and land class values to matching OSM key value combinations and drop and rows that don't match.
df_landcover_samples["landcover"] = df_landcover_samples["keyvalue"].map(land_covers)
df_landcover_samples["landclass"] = df_landcover_samples["landcover"].map(land_classes)
df_landcover_samples = df_landcover_samples.dropna()
df_landcover_samples.isnull().sum()
df_null = df_landcover_samples[df_landcover_samples.isna().any(axis=1)]
df_landcover_samples.to_csv('gbq_landcover.csv')
assert df_null.size == 0, "DataFrame df_null should have a size of 0"


In [ ]:
df_landcover_samples.head()
df_landcover_samples.describe(include='all')

In [ ]:
fig = plt.subplots(figsize=(9, 6))
sns.factorplot("landcover", data=df_landcover_samples, aspect=2, kind="count", color='steelblue')
plt.title('OpenStreetMap Feature Counts', fontsize=14) 
plt.tight_layout()
plt.savefig(os.path.join(figures_save_location,"osm_count" + "_by_type"),dpi=300)

## Export Data to CSV for Backup

In [ ]:
df_landcover_samples.to_csv('gbq_landcover.csv')

In [ ]:
df_landcover_samples.head()
df_landcover_samples['date'] = pd.to_datetime(df_landcover_samples.osm_timestamp,format='%Y%m', errors='coerce')
df_landcover_samples['month_year'] = pd.to_datetime(df_landcover_samples['date']).dt.to_period('M')
df_landcover_samples['count'] = 1
df_landcover_samples.head()

df_osm_temporal = df_landcover_samples.groupby(by=["month_year",'landcover'], as_index=False).count()
df_osm_temporal = df_osm_temporal.pivot( "month_year","landcover", "count")
df_osm_temporal = df_osm_temporal.resample('1M').count().fillna(0)
ax = df_osm_temporal.plot(kind='area')
ax.set_xlabel("Year")
ax.set_ylabel("Count")
plt.savefig(os.path.join(figures_save_location,"osm_year_count" + "_by_type"),dpi=300)
# sns.heatmap(df_osm_temporal)
# sns.scatterplot(data=df_osm_temporal)

# df_osm_temporal.index
# sns.relplot(x="month_year", y="count", hue="keyvalue", data=df_osm_temporal);
# sns.heatmap(df_osm_temporal.pivot("keyvalue", "month_year", "count"), annot=True, fmt="f")
# sns.kdeplot(data=df_osm_temporal, x=date, y=count, levels=5, color=keyvalue, linewidths=1)
# df_osm_temporal = df_osm_temporal.set_index("month_year")
# sns.scatterplot(data=df_osm_temporal,  y="count", hue="keyvalue")



#### Convert raw OSM data to geojson so that it can be transformed into a GEE FeatureCollection.

Buffer road data to match road size

In [ ]:
"""
Convert raw OSM data to geojson so that it can be transformed into a GEE FeatureCollection.
"""
sample_features = []
road_features = []
water_features = []
for index, row in df_landcover_samples.iterrows():
  geojson = json.loads(row["geom"])

  # if row["key"] != "highway" and geojson["type"] != "Point" and len(geojson["coordinates"]) > 2:
  #   geojson["type"] = "Polygon"
  #   first_node = geojson["coordinates"][0]
  #   node_len = len(geojson["coordinates"]) - 1
   
  #   coordinates =[geojson["coordinates"]]
   
  #   geojson["coordinates"] = coordinates

  feature = ee.Feature(geojson,{"landcover": row["landcover"], "landclass": row["landclass"]})
  
  #Values deriverd from https://safety.fhwa.dot.gov/geometric/pubs/mitigationstrategies/chapter3/3_lanewidth.cfm and 
  # https://wiki.openstreetmap.org/wiki/Highway_classes
  if row["key"] == "highway" and geojson["type"] == "LineString":
  #   if row['value'] == 'residential':
  #     feature = feature.buffer(2.7 * 1)
  #   elif row['value'] == 'unclassifed':
  #      feature = feature.buffer(2.7 * 1)
  #   elif row['value'] == 'tertiary': #new
  #     feature = feature.buffer(2.7 * 2)
  #   elif row['value'] == 'secondary': #new
  #      feature = feature.buffer(3.6 * 2)
  #   elif row['value'] == 'primary':
  #      feature = feature.buffer(3.6 * 2)
  #   elif row['value'] == 'trunk':
  #      feature = feature.buffer(3.6 * 4)
  #   elif row['value'] == 'motorway':
  #      feature = feature.buffer(3.6 * 4)
  #   elif row['value'] == 'motorway_link':
  #      feature = feature.buffer(3.6 * 1)
  #   elif row['value'] == 'trunk_link':
  #      feature = feature.buffer(3.6 * 1)
  #   elif row['value'] == 'primary_link':
  #      feature = feature.buffer(3.6 * 1)
  #   elif row['value'] == 'secondary_link':
  #      feature = feature.buffer(3.6 * 1)
  #   elif row['value'] == 'tertiary_link':
  #      feature = feature.buffer(3.6 * 1)

    road_features.append(feature)
  elif row["value"] == "water":
    water_features.append(feature)
  else:
    sample_features.append(feature)
 


print("Sample Features {}".format(len(sample_features)))
print("Road Features {}".format(len(road_features)))
print("Water Features {}".format(len(water_features)))
sample_feature_collection = ee.FeatureCollection(sample_features[0:(int(len(sample_features)/4))])
water_feature_collection = ee.FeatureCollection(water_features)
# print(sample_feature_collection.size())
road_feature_collection = ee.FeatureCollection(road_features)
road_class = ee.Image().byte().paint(road_feature_collection, "landclass").rename("landclass")
# classes = sample_feature_collection.filter(ee.Filter.notNull(['landclass'])).reduceToImage(properties=["landclass"], reducer=ee.Reducer.first())
classes = ee.Image().byte().paint(sample_feature_collection, "landclass").rename("landclass")

##Load Planet ImageColelction

In [ ]:
# Fetch Planet Data.

start_image = '2018-01-30T15:21:54'
end_image = '2019-12-31T15:38:18'
start_image = "2018-06-01" 
end_image = "2020-08-26" 
# planet_scope = ee.ImageCollection('users/ctwhite/planetdata/rapid_dsm_aoi').filterDate(start_image, end_image).filterBounds(aoi) #Auto generated
print("ImageCollection: ", imageCollection) 
planet_scope_ic = ee.ImageCollection(imageCollection)
print("ImageCollection Size: ", planet_scope_ic.size().getInfo())

planet_scope = (planet_scope_ic
  .filterBounds(aoi)
  .filterDate(start_image, end_image)
  .filter(ee.Filter.gt('heavy_haze_percent', 0).Not())
  .filter(ee.Filter.gt('light_haze_percent', 0).Not())
  .filter(ee.Filter.eq('cloud_percent', 0)))

print("ImageCollection Filtered Size: ", planet_scope.size().getInfo())
  

## Export OSM Data

In [ ]:
myMap = folium.Map(location=center_map, zoom_start=12, height=500)
planet_scope_mosaic_log = planet_scope.median().log()
planet_scope_mosaic_vis_params = {"bands": ['b3','b2','b1'], 'min':5.67, 'max': 8.39}
myMap.add_ee_layer(planet_scope_mosaic_log, planet_scope_mosaic_vis_params, 'PlanetScope Log Median')
classes = sample_feature_collection.filterBounds(aoi).filter(ee.Filter.notNull(['landclass'])).reduceToImage(properties=["landclass"], reducer=ee.Reducer.first()).clipToCollection(aoi)
road_rast = road_feature_collection.filterBounds(aoi).filter(ee.Filter.notNull(['landclass'])).reduceToImage(properties=["landclass"], reducer=ee.Reducer.first()).clipToCollection(aoi)
water_class = water_feature_collection.filterBounds(aoi).filter(ee.Filter.notNull(['landclass'])).reduceToImage(properties=["landclass"], reducer=ee.Reducer.first()).clipToCollection(aoi)
aoi_rast = ee.Image().byte().paint(aoi)

aoi_mask_tmp = aoi_rast.eq(1)
aoi_mask = aoi_mask_tmp.Not()
aoi_mask = aoi_mask.mask(aoi_mask)
myMap.add_ee_layer(classes, {"min": 0, "max": 6, "palette": landcover_color_palette}, 'Trainging Classes ')
# myMap.add_ee_layer(road_feature_collection.draw(color= 'blue', strokeWidth= 2), {}, 'Roads')

myMap.add_ee_layer(water_class.clipToCollection(aoi), {"min": 0, "max": 6, "palette": landcover_color_palette}, 'Water Classes ')

myMap.add_ee_layer(aoi_mask, {"palette": ["blue"], "opacity": 0.25}, 'AOI Mask')

myMap.add_child(folium.LayerControl())

display(myMap)

In [ ]:
sns.set_palette(sns.color_palette([ "#%s" % c if c is not "purple" else c for c in landcover_color_palette]))
# sns.barplot(x="landclass", y="freq",hue='landclass', data=df_stratified_sample)

In [ ]:
myMap = folium.Map(location=center_map, zoom_start=12, height=500)
planet_scope_mosaic_log = planet_scope.median().log()
planet_scope_mosaic_vis_params = {"bands": ['b3','b2','b1'], 'min':5.67, 'max': 8.39}
myMap.add_ee_layer(planet_scope_mosaic_log, planet_scope_mosaic_vis_params, 'PlanetScope Log Median')
classes_rast = sample_feature_collection.filter(ee.Filter.notNull(['landclass'])).reduceToImage(properties=["landclass"], reducer=ee.Reducer.first())
aoi_rast = ee.Image().byte().paint(aoi)

# Removed water for sample data raster
aoi_mask_tmp = aoi_rast.eq(1)
aoi_mask = aoi_mask_tmp.Not()
aoi_mask = aoi_mask.mask(aoi_mask)
myMap.add_ee_layer(classes_rast.updateMask(aoi_mask), {"min": 0, "max": 6, "palette": landcover_color_palette}, 'Sample Features')
myMap.add_ee_layer(aoi_mask, {"palette": ["blue"], "opacity": 0.25}, 'AOI Mask')

myMap.add_child(folium.LayerControl())

display(myMap)


In [ ]:
#https://www.mrlc.gov/data/legends/national-land-cover-database-2016-nlcd2016-legend


## Add Bare Earth Samples 
Derived from NDVI and BSI

In [ ]:
ps_median = planet_scope.median().clipToCollection(aoi_bbox)

bsiVisParam = {"bands":["bsi"],"min":99.543,"max":103.578,"palette":["ff3e04","ffb308","ffec08","ffffff"],'dimensions': 1000};
# Image(url=bsi.clip(aoi).getThumbUrl(bsiVisParam))

# bare_earth_bsi = bsi.gte(103).And(bsi.lte(105.5)).And(ndvi.gt(0.16)).And(ndvi.lt(0.22)).selfMask()
classes_mask = ps_median.where(classes.gt(0), -9999).neq(-9999).selfMask()
ndvi = createNDVI(ps_median) #.updateMask(classes_mask)
bsi = createBSI(ps_median) #.updateMask(classes_mask)
ndwi = createNDWI(ps_median) #.updateMask(classes_mask)
dsbi = createDSBI(ps_median)


#experimental index to seperate grass, forest, and impervious
# ndbsvi = createBSI_NDVI_index(bsi, ndvi)


bare_earth_bsi = (
    bsi.gt(103.5)
      .And(bsi.lte(107))
      .And(ndvi.gt(0.15))
      .And(ndvi.lt(0.2))
      # .And(ndwi.lt(0)) #Filter out water features
      .selfMask())

objectId = bare_earth_bsi.connectedComponents(ee.Kernel.plus(1), 128)
# Compute the number of pixels in each object defined by the "labels" band.
objectSize = objectId.select('labels').connectedPixelCount(128,False)
# Get a pixel area image.
pixelArea = ee.Image.pixelArea()
# Multiply pixel area by the number of pixels in an object to calculate
# the object area. The result is an image where each pixel
# of an object relates the area of the object in m^2.
objectArea = objectSize.multiply(pixelArea)
# Map.addLayer(objectArea, null, 'objectArea');

## Threshold the `objectArea` image to define a mask that will mask out
areaMask = objectArea #.gte(5);

# Update the mask of the `objectId` layer defined previously using the
# minimum area mask just defined.
bare_earth_samples = objectId.updateMask(classes_mask.select('b1')) #.updateMask(areaMask)
bare_earth_vector = bare_earth_samples.select('labels').reduceToVectors(geometry=aoi,labelProperty='labels',scale=30 )


# center_map = [35.752265080410844,-78.63944155399622]



# Create a folium map object.
print(center_map)
myMap = folium.Map(location=center_map, zoom_start=14, height=500)
planet_scope_mosaic_log = planet_scope.median().log().clipToCollection(aoi_bbox)
planet_scope_mosaic_vis_params = {"bands": ['b3','b2','b1'], 'min':5.67, 'max': 8.39}
myMap.add_ee_layer(planet_scope_mosaic_log, planet_scope_mosaic_vis_params, 'PlanetScope Log Median')

myMap.add_ee_layer(bsi.gt(104).And(bsi.lt(105)).selfMask(), {"palette": ["yellow"],"bands":["bsi"],"min":102.7,"max":105,"opacity": 0.9}, 'BSI')
myMap.add_ee_layer(ndvi.gt(0.25).And(ndvi.lt(0.27)).selfMask(), {"palette": ["red"], "opacity": 0.5}, 'NDVI')






# print(bare_earth_samples.bandNames().getInfo())
myMap.add_ee_layer(bare_earth_samples.select(['labels']), {"palette": ['FF0000']}, 'Large hotspots')
myMap.add_child(folium.LayerControl())

display(myMap)


In [ ]:
wake_building_permits_url = 'https://opendata.arcgis.com/datasets/5b8394817ec34097819534e67ccd4d8b_0.geojson'
gdf_building_permits = gpd.read_file(wake_building_permits_url)



In [ ]:
# gdf_building_permits.plot()
gdf_building_permits.describe()
gdf_building_permits_filterd = gdf_building_permits[
              (gdf_building_permits['PERMIT_STATUS'] == 'Complete') & 
              (gdf_building_permits['ISSUE_DATE']  >= "2017-01-01") & 
              (gdf_building_permits['FINALED_DATE']  <= end_image) & 
              # (gdf_building_permits['SQUARE_FEET']  >= 5000) &
              (gdf_building_permits['WORK_CLASS']  == 'New Building or Addition') &
              (gdf_building_permits['geometry'].isna() == False)
              ]
gdf_building_permits_filterd.describe()
gdf_building_permits_filterd.head(100)


In [ ]:
gdf_building_permits_filterd.WORK_CLASS.unique()


In [ ]:
building_permit_features = []
for i, r in gdf_building_permits_filterd.iterrows():
  if r.empty is not True:
    tmp_dict = r.to_dict()
    del tmp_dict['geometry']
    # print(tmp_dict)
    # del tmp_dict['const']
    # Square Feet field is not up to date so it does catch all constructions
    # Set buffer value to 100m so it is more visible
    buff_dist = (((r.SQUARE_FEET / 2.0) / 3.281) / 1000.0)
    print(f"SqFt: {r.SQUARE_FEET}, Buff Dist: {buff_dist}")
    feature = ee.Feature(ee.Geometry.Point(r.X, r.Y), {'sqft': r.SQUARE_FEET,'buf_dist':100,  'permit_type': r.PERMIT_TYPE, 'issue_data':r.ISSUE_DATE, 'work_class': r.WORK_CLASS})
    building_permit_features.append(feature)
    

building_permits_feature_collection = ee.FeatureCollection(building_permit_features)

## Export Before after Images to Google Cloud Storage



In [ ]:

# before_image = planet_scope.filterDate("2020-04-01", "2020-04-30").median()
# after_image = planet_scope.filterDate("2020-05-01", "2020-05-31").median()

# before_image = planet_scope.filterDate('2018-06-01', '2018-12-31').median().clip(aoi)
# after_image = planet_scope.filterDate('2019-01-01', '2019-03-01').median().clip(aoi)
# 2018-12-18T15:05:08 to 2019-02-26T14:54:21
# Hardcoded for testing 
start_image = "2018-06-01" 
end_image = "2020-08-26"

from datetime import datetime
date_format = "%Y-%m-%d"
# date_format = "%Y-%m-%dT%H:%M:%S"

d1 = datetime.strptime(start_image,date_format)
d2 = datetime.strptime(end_image,date_format)
# date1 + (date2 - date1) / 2
date_midpoint = d1 + (d2 - d1) / 2 # first dat
print(date_midpoint.isoformat())
print(date_midpoint)
before_img_col = planet_scope.filterBounds(aoi).filterDate(start_image, date_midpoint)
after_img_col = planet_scope.filterBounds(aoi).filterDate(date_midpoint, end_image)
print("ImageCollection Before Size: ", before_img_col.size().getInfo())
print("ImageCollection After Size: ", after_img_col.size().getInfo())

before_image = before_img_col.median().toFloat().clipToCollection(aoi_bbox)
after_image = after_img_col.median().toFloat().clipToCollection(aoi_bbox)

# Try mosaic over median
# before_image = before_img_col.mosaic().toFloat().clipToCollection(aoi_bbox)
# after_image = after_img_col.mosaic().toFloat().clipToCollection(aoi_bbox)

print(after_image.getInfo())

# before_start_date = before_image.getInfo()
# print(before_start_date)
ps_collection_obcd = {
    "before": before_image,
    "after": after_image
}

In [ ]:
exportToDrive(before_image,"ps_before_3m", resolution=3)
exportToDrive(after_image,"ps_after_3m", resolution=3)

In [ ]:
myMap = folium.Map(location=center_map, zoom_start=12, height=500)
planet_scope_mosaic_log = planet_scope.median().log()
planet_scope_mosaic_vis_params = {"bands": ['b3','b2','b1'], 'min':5.67, 'max': 8.39}
myMap.add_ee_layer(before_image.log(), planet_scope_mosaic_vis_params, 'PlanetScope Log Median')
buffered_building_permits_feature_collection = building_permits_feature_collection.map(lambda f: f.buffer(f.get('buf_dist')))
building_permits_rast = ee.Image().byte().paint(buffered_building_permits_feature_collection)

myMap.add_ee_layer(building_permits_rast, {"palette": ["yellow"], "opacity": 0.60}, 'Building Permits')
myMap.add_ee_layer(aoi_mask, {"palette": ["blue"], "opacity": 0.25}, 'AOI Mask')

myMap.add_child(folium.LayerControl())

display(myMap)


## Testing Statistical Threashold Classification

In [ ]:
ndvi = createNDVI(ps_median)
bsi = createBSI(ps_median)
dsbi = createDSBI(ps_median)
#experimental index to seperate grass, forest, and impervious
ndbsvi = createBSI_NDVI_index(bsi, ndvi)
count = 0

def quantileIndexes(image, band, palette=["ff3e04","ffb308","ffec08","ffffff"]):
  percentiles = image.reduceRegion(
        reducer=ee.Reducer.percentile([10,25,50,75,90]),
        geometry=aoi,
        scale=30,
        # tileScale=4,
        maxPixels= 1e12)


  percentile10th = ee.Number(percentiles.get(band+'_p10'))
  lowerQuartile = ee.Number(percentiles.get(band+'_p25'))
  median = ee.Number(percentiles.get(band+'_p50'))
  upperQuartile = ee.Number(percentiles.get(band+'_p75'))
  percentile90th = ee.Number(percentiles.get(band+'_p90'))
  print("10th: {}".format(percentile10th.getInfo()))
  print("lower: {}".format(lowerQuartile.getInfo()))
  print("median: {}".format(median.getInfo()))
  print("upper: {}".format(upperQuartile.getInfo()))
  print("90th: {}".format(percentile90th.getInfo()))

  # Get AOI centroid
  # center_map = aoi.geometry().centroid().getInfo()['coordinates'].reverse()
  myMap = folium.Map(location=center_map, zoom_start=16, height=500)
  planet_scope_mosaic_log = planet_scope.median().log().clip(aoi)
  planet_scope_mosaic_vis_params = {"bands": ['b3','b2','b1'], 'min':5.67, 'max': 8.39}
  myMap.add_ee_layer(planet_scope_mosaic_log, planet_scope_mosaic_vis_params, 'PlanetScope Log Median')

  
  
  #Low to High
  myMap.add_ee_layer(image, {"bands":[band],"min":lowerQuartile.getInfo(),"max":upperQuartile,"palette":palette}, '%s low to High' %band)
  #Low to Med
  myMap.add_ee_layer(image, {"bands":[band],"min":lowerQuartile.getInfo(),"max":median.getInfo(),"palette":palette}, '%s low to Med' % band)
  #Med to High
  myMap.add_ee_layer(image, {"bands":[band],"min":median.getInfo(),"max":upperQuartile.getInfo(),"palette":palette}, '%s Med to High' % band)
  #low
  myMap.add_ee_layer(image, {"bands":[band],"max":lowerQuartile.getInfo(),"palette":palette}, '%s low' % band)
  #high
  myMap.add_ee_layer(image, {"bands":[band],"min":upperQuartile.getInfo(),"palette":palette}, '%s high' % band)

  #masks
  # myMap.add_ee_layer(water,{},"Water Mask")

  
  landcover_color_palette = [
    'E29E8C', # Class 0 - road/highways         NLCD class color 22
    'B50000', # Class 1 - Building dark red     NLCD class color 24
    'D2CDC0', # Class 2 - barren                NLCD class color 31
    '38814E', # Class 3 - Forest                NLCD class color 42
    '85C77E', # Class 4 - Grass                 NLCD class color 41
    '5475A8', # Class 5 -  water                 NLCD class color 11
    'E8D1D1' # Class 5 -  developed             NLCD class color 21
]

  dem = ee.Image("USGS/NED").select('elevation')
  dsm = ee.Image("JAXA/ALOS/AW3D30/V2_2").select('AVE_DSM')
  buildings_canopy = dsm.subtract(dem) #.focal_min(radius=1)

  # Less Than or equal to Lower
  myMap.add_ee_layer(image.updateMask(image.lte(lowerQuartile)), {"bands":[band],"palette":[landcover_color_palette[0]],"max": lowerQuartile.getInfo()}, '%s Less Than or equal to Lower (road/highways)' % band)

  # Less Than or equal to 10th
  myMap.add_ee_layer(image.updateMask(image.lte(percentile10th)), {"bands":[band],"palette":["grey"],"max": percentile10th.getInfo()}, '%s Less Than or equal to 10th (GREY) - Developed Imperious Road/Building/Asphalt' % band)

  # Less Than median greater than lower
  myMap.add_ee_layer(image.updateMask(image.gt(lowerQuartile).And(image.lt(median))), {"bands":[band],"min":lowerQuartile.getInfo(),"max":median.getInfo(),"palette":[landcover_color_palette[4]]}, '%s Less Than median greater than lower mask (GRASS)' % band)

  # Greater Than Lower And Less than Upper
  myMap.add_ee_layer(image.updateMask(image.gt(lowerQuartile).And(image.lt(upperQuartile))), {"bands":[band],"min":lowerQuartile.getInfo(),"max":upperQuartile.getInfo(),"palette":[landcover_color_palette[4]]}, '%s Greater Than Lower And Less than Upper (Grass/Forest)' % band)

  # Greater Than median less than upper
  myMap.add_ee_layer(image.updateMask(image.gt(median).And(image.lt(upperQuartile))), {"bands":[band],"min":median.getInfo(),"max":upperQuartile.getInfo(),"palette":[landcover_color_palette[3]]}, '%s Greater Than median less than upper mask (Forest Boundary)' % band) 

  # Greater Than or equal to Upper
  myMap.add_ee_layer(image.updateMask(image.gte(upperQuartile)), {"bands":[band],"palette":[landcover_color_palette[3]],"min":upperQuartile.getInfo()}, '%s Greater Than or equal to Upper (Forest)' % band)

  # Greater Than or equal to percentile90th
  myMap.add_ee_layer(image.updateMask(image.gte(percentile90th)), {"bands":[band],"palette":[landcover_color_palette[3]],"min":percentile90th.getInfo()}, '%s Greater Than or equal to 90th (Forest)' % band)


  # Grass DSM
  myMap.add_ee_layer(image.updateMask(image.gt(lowerQuartile).And(buildings_canopy.lte(2))), {"bands":[band],"min":lowerQuartile.getInfo(),"max":upperQuartile.getInfo(),"palette":[landcover_color_palette[4]]}, '%s Grass with DSM)' % band)
  # Forest DSM
  myMap.add_ee_layer(image.updateMask(image.gt(lowerQuartile).And(buildings_canopy.gt(2))), {"bands":[band],"min":lowerQuartile.getInfo(),"max":upperQuartile.getInfo(),"palette":[landcover_color_palette[3]]}, '%s Forest with DSM)' % band)
   
  # Developed DSM
  myMap.add_ee_layer(image.updateMask(image.lte(lowerQuartile).And(buildings_canopy.lte(3))), {"bands":[band],"palette":[landcover_color_palette[0]],"max": percentile10th.getInfo()}, '%s Developed with DSM' % band)

  # Building DSM
  myMap.add_ee_layer(image.updateMask(image.lte(lowerQuartile).And(buildings_canopy.gt(3))), {"bands":[band],"palette":[landcover_color_palette[1]],"max": percentile10th.getInfo()}, '%s Buildings with DSM' % band)

  
 
  myMap.add_child(folium.LayerControl())
  display(myMap)
  

# quantileIndexes(ndbsvi, 'ndbsiv')




In [ ]:
quantileIndexes(ndvi.clipToCollection(aoi), 'ndvi', ['green','white', 'purple'])


## Bare Earth Merge

In [ ]:
bare_earth_feature_collection = bare_earth_vector.map(lambda f: f.set({"landclass": 2, "landcover": "barren"}))
combined_feature_collection = sample_feature_collection.merge(bare_earth_feature_collection).map(lambda f: f.set("area", f.area(0.001)));

# Using Just OSM DATA
# combined_feature_collection = sample_feature_collection.map(lambda f: f.set("area", f.area(0.001)));

# print(bare_earth_vector.getInfo())

fc_list = combined_feature_collection.toList(count=1500).getInfo()
list_of_properties = [f['properties'] for f in fc_list]
df_combined_samples = pd.DataFrame(list_of_properties)

# # Get a list of sample sizes
# stratifed_samples_sizes = df_sample_stats['sample_size'].astype(int).values.tolist()

# # Set water class to 0 because we will copy this data in after classification
# stratifed_samples_sizes[5] = 0 # Remove Water
# # stratifed_samples_sizes[2] = 19
# # stratifed_samples_sizes[0] = 0 #roads added post classification

# print(stratifed_samples_sizes)


In [ ]:
df_sample_stats = df_combined_samples.groupby(['landcover','landclass'])['area'].describe().reset_index()
df_sample_stats['area']= df_combined_samples.groupby(['landcover','landclass'])['area'].sum().values
df_sample_stats.sort_values('landclass', inplace=True)
print(df_sample_stats)
#drop the roads row by index
# df_sample_stats.drop(5, inplace=True) 
print(df_sample_stats[df_sample_stats['landclass'] == 2]['50%'].head(20))

#Get min area
min_area = df_sample_stats['area'].min()
print(min_area)

# calcluate the sample size by dividing the minmum total class area and dividing it by the median object area feature class.
df_sample_stats['sample_size'] = df_sample_stats['50%'].apply(lambda x: int(min_area / x) if int(min_area / x) else 2, 1)

df_sample_stats['median'] = df_sample_stats['50%']

# Reformating for paper
df_sample_stats[['landcover','count','min', 'median', 'sample_size']].set_index('landcover')
# Get a list of sample sizes
stratifed_samples_sizes = df_sample_stats['sample_size'].astype(int).values.tolist()

# Set water class to 0 because we will copy this data in after classification
#stratifed_samples_sizes[5] = 0 # Remove Water
# stratifed_samples_sizes[2] = 19
# stratifed_samples_sizes[0] = 0 #roads added post classification

print(stratifed_samples_sizes)


In [ ]:
# Get a list of sample sizes
stratifed_samples_sizes = df_sample_stats['sample_size'].astype(int).values.tolist()

# Set water class to 0 because we will copy this data in after classification
#stratifed_samples_sizes[5] = 0 # Remove Water
# stratifed_samples_sizes[2] = 19
# stratifed_samples_sizes[0] = 0 #roads added post classification

print(stratifed_samples_sizes)

In [ ]:
f, ax = plt.subplots(figsize=(12, 8))
ax.set_xscale("log")
df_combined_samples['area_km'] = df_combined_samples['area'].apply(lambda x: x / 1e6)
# df_combined_samples =  df_combined_samples[df_combined_samples['landcover'] != 'road']
# Plot the orbital period with horizontal boxes
sns.boxplot(x="area", y="landcover", data=df_combined_samples.sort_values('landclass'),
            whis=[0, 100],palette=sns.color_palette(['#B50000','#D2CDC0', '#38814E','#85C77E', '#E8D1D1']))

# Add in points to show each observation
sns.stripplot(x="area", y="landcover", data=df_combined_samples.sort_values('landclass'),
              size=2, color=".3", linewidth=0, dodge=True)

# Tweak the visual presentation
ax.xaxis.grid(True)
ax.set(ylabel="")
ax.set_xlabel("Area m2",fontsize=16 )
ax.tick_params(labelsize=16)
sns.despine(trim=True, left=True)
# ax.axvline(x=df_sample_stats[df_sample_stats['landcover'] == 'road']['50%'].values, linewidth=1, color="#"+landcover_color_palette[0],linestyle="--", label="Road")
# ax.axvline(x=df_sample_stats[df_sample_stats['landcover'] == 'building']['50%'].values, linewidth=1, color="#"+landcover_color_palette[1],linestyle="--", label="Building")
# ax.axvline(x=df_sample_stats[df_sample_stats['landcover'] == 'barren']['50%'].values, linewidth=1, color="#"+landcover_color_palette[2],linestyle="--", label="Barren")
# ax.axvline(x=df_sample_stats[df_sample_stats['landcover'] == 'forest']['50%'].values, linewidth=1, color="#"+landcover_color_palette[3],linestyle="--", label="Forest")
# ax.axvline(x=df_sample_stats[df_sample_stats['landcover'] == 'grass']['50%'].values, linewidth=1, color="#"+landcover_color_palette[4],linestyle="--", label="Grass")
# ax.axvline(x=df_sample_stats[df_sample_stats['landcover'] == 'water']['50%'].values, linewidth=1, color="#"+landcover_color_palette[5],linestyle="--", label="Water")
plt.title('Land Cover Samples', fontsize=24) 
# landcover_color_palette = [
#     'E29E8C', # Class 0 - road/highways         NLCD class color 22
#     'B50000', # Class 1 - Building dark red     NLCD class color 24
#     'D2CDC0', # Class 2 - barren                NLCD class color 31
#     '38814E', # Class 3 - Forest                NLCD class color 42
#     '85C77E', # Class 4 - Grass                 NLCD class color 41
#     '5475A8', # Class 5 -  water                 NLCD class color 11
#     'E8D1D1' # Class 5 -  developed             NLCD class color 21
# ]

plt.savefig(os.path.join(figures_save_location,"Samples Box Area Plot"),dpi=300)

In [ ]:
# df_sample_stats.rename(columns=['count', 'mean', 'median', 'area', 'sample_size']).set_index('landcover')[['count', 'mean', '50%', 'area', 'sample_size (median)']]

In [ ]:
classes_w_barren = ee.Image().byte().paint(combined_feature_collection, "landclass").rename("landclass").where(
    classes.eq(5).selfMask(), 5).where(
        classes.eq(0).selfMask(), 0).where(
            classes.eq(1).selfMask(), 1
        )

#classes_w_barren = classes.where(bare_earth_samples.eq(2).And(classes.Not()), 2).clipToCollection(aoi_bbox)
      


## Outlire Detection

In [ ]:
#https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/9ZIUGQ

def detectOutlires(image, band, scale=3, region=aoi):
  # // Tukey's lower and upper fence
  percentiles = image.reduceRegion(
      reducer=ee.Reducer.percentile([10,25,50,75,90]),
      geometry=region,
      scale=scale,
      maxPixels= 1e12)
  
  lowerQuartile = ee.Number(percentiles.get(band+'_p25'))
  median = ee.Number(percentiles.get(band+'_p50'))
  upperQuartile = ee.Number(percentiles.get(band+'_p75'))

  IQR = upperQuartile.subtract(lowerQuartile)
  lowerFence = lowerQuartile.subtract(IQR.multiply(1.5))
  upperFence = upperQuartile.add(IQR.multiply(1.5))
  
  quartiles = image.gt(lowerQuartile).add(image.gt(median)).add(image.gt(upperQuartile)).remap([0,1,2,3],[1,2,3,4]).rename('quartile')
  outliers = image.gt(lowerFence).add(image.gt(upperFence)).remap([0,1,2],[1,0,2])
  tukeys = outliers.updateMask(outliers.neq(0)).rename('fence')
  
  # // Z-score
  mean = ee.Number(image.reduceRegion(
      reducer= ee.Reducer.mean(),
      geometry= region,
      scale= scale,
      maxPixels= 1e12).get(band))

  stdDev = ee.Number(image.reduceRegion(
      reducer= ee.Reducer.stdDev(),
      geometry= region,
      scale= scale,
      maxPixels= 1e12).get(band));
  
  zScore = image.subtract(mean).divide(stdDev).rename('zscore')
  
  # // Modified Z-score
  medAbsDev = image.subtract(median).abs();
  
  medianMedAbsDev = ee.Number(medAbsDev.reduceRegion(
      reducer= ee.Reducer.median(),
      geometry= region,
      scale= scale,
      maxPixels= 1e12).get(band))
  
  zScoreMod = image.subtract(mean).multiply(0.6745).divide(medianMedAbsDev).abs().rename('zscore_mod')
  zScoreModExp = zScoreMod.gt(3.5).multiply(3)
  zScoreModOutliers = zScoreModExp.updateMask(zScoreModExp.eq(3)).rename('zmod_outlier')
  
  # // Lower fence/upper fance + modified s-score outliers
  combined = zScoreModExp.add(outliers).rename('combined_outliers').remap([0,1,2,3,4,5],[0,3,4,5,1,2])
                #  // 0 = none
                #  // 1 = lower fence
                #  // 2 = upper fence
                #  // 3 = modified z-score
                #  // 4 = lower fence + modified z-score
                #  // 5 = upper fence + modified z-score 
  combinedMask = combined.updateMask(combined.neq(0))
      
  # // Geary's C statistic
  clist = [1, 1, 1, 1, 1, 1, 1, 1, 1]
  centerList = [1, 1, 1, 1, 0, 1, 1, 1, 1]
  lists = [clist, clist, clist, clist, centerList, clist, clist, clist, clist]
  kernel = ee.Kernel.fixed(9, 9, lists, -4, -4, False)
  neighs = image.neighborhoodToBands(kernel)
  # import math
  gearys = image.subtract(neighs).pow(2).reduce(ee.Reducer.sum()).divide(9**2)
         
  gearysQuartiles = gearys.reduceRegion(
      reducer=ee.Reducer.percentile([10,25,75,90]),
      geometry= region,
      scale= scale,
      maxPixels= 1e12)
  
  # gearys10 = gearysQuartiles.get('sum_p10').getInfo()
  # print(gearys10)

  gearysLowerQuartile = ee.Number(gearysQuartiles.get('sum_p25'))
  # print(gearysLowerQuartile)
  gearysUpperQuartile = ee.Number(gearysQuartiles.get('sum_p75'))
  gearysIQR = gearysUpperQuartile.subtract(gearysLowerQuartile)
  gearysUpperFence = gearysUpperQuartile.add(gearysIQR.multiply(1.5))
  gearysAccum = gearys.gt(gearysUpperFence).rename('gearys_outlier')
  print('gearysAccum')
  gearysOutlier = gearysAccum.updateMask(gearysAccum.eq(1)).rename('spatial_outlier')
  print('gearysOutlier')
  
  # // Locate pixels ≤ zero
  lteZero = image.lte(0).updateMask(image.lte(0).eq(1))
  print('lteZero')

  # // Layer design values
  zScorePercentiles = zScore.reduceRegion(
      reducer=ee.Reducer.percentile([10,90]),
      geometry= region,
      scale= scale,
      maxPixels= 1e12)
  print('zScorePercentiles')

  zScoreModPercentiles = zScoreMod.reduceRegion(
      reducer=ee.Reducer.percentile([10,90]),
      geometry= region,
      scale= scale,
      maxPixels= 1e12)
  
  print('zScoreModPercentiles')

  raw10 = ee.Number(percentiles.get(band+'_p10')).getInfo()
  print('raw10')
  raw90 = ee.Number(percentiles.get(band+'_p90')).getInfo()
  print('raw90')
  zScore10 = ee.Number(zScorePercentiles.get('zscore_p10')).getInfo()
  print('zScore10')
  zScore90 = ee.Number(zScorePercentiles.get('zscore_p90')).getInfo()
  print('zScore90')
  zScoreMod10 = ee.Number(zScoreModPercentiles.get('zscore_mod_p10')).getInfo()
  print('zScoreMod10')
  zScoreMod90 = ee.Number(zScoreModPercentiles.get('zscore_mod_p90')).getInfo()
  print('zScoreMod90')
  gearys10 = ee.Number(gearysQuartiles.get('sum_p10')).getInfo()
  print('gearys10')
  gearys90 = ee.Number(gearysQuartiles.get('sum_p90')).getInfo()
  print('gearys90')

  
 
  # -- Make sure to change the scale to calculate accurate area percentages
  print("AreaImage Start")
  areaImage = image.multiply(0).rename('area')
  print("AreaImage End")
  totalArea = ee.Number(areaImage.add(1).reduceRegion(
      reducer= ee.Reducer.sum(),
      geometry= region,
      scale= scale,
      maxPixels= 1e12).get('area'))

  print("totalArea")

  lfmzArea = ee.Number(areaImage.add(combinedMask.eq(1)).reduceRegion(
      reducer= ee.Reducer.sum(),
      geometry= region,
      scale= scale,
      maxPixels= 1e12).get('area')).divide(totalArea).multiply(100).getInfo() #.toFixed(2) #round(a, 2)
  print("lfmzArea")
  ufmzArea = ee.Number(areaImage.add(combinedMask.eq(2)).reduceRegion(
      reducer= ee.Reducer.sum(),
      geometry= region,
      scale= scale,
      maxPixels= 1e12).get('area')).divide(totalArea).multiply(100).getInfo() #.toFixed(2)
  print("ufmzArea")
  lfArea = ee.Number(areaImage.add(tukeys.eq(1)).reduceRegion(
      reducer= ee.Reducer.sum(),
      geometry= region,
      scale= scale,
      maxPixels= 1e12).get('area')).divide(totalArea).multiply(100).getInfo() #.toFixed(2)
  print("lfArea")

  ufArea = ee.Number(areaImage.add(tukeys.eq(2)).reduceRegion(
      reducer= ee.Reducer.sum(),
      geometry= region,
      scale= scale,
      maxPixels= 1e12).get('area')).divide(totalArea).multiply(100).getInfo() #.toFixed(2)
  print("ufArea")

  mzArea = ee.Number(areaImage.add(zScoreModOutliers.eq(3)).reduceRegion(
      reducer= ee.Reducer.sum(),
      geometry= region,
      scale= scale,
      maxPixels= 1e12).get('area')).divide(totalArea).multiply(100).getInfo() #.toFixed(2)
  
  lteZeroArea = ee.Number(areaImage.add(lteZero.eq(0)).reduceRegion(
      reducer= ee.Reducer.sum(),
      geometry= region,
      scale= scale,
      maxPixels= 1e12).get('area')).divide(totalArea).multiply(100).getInfo() #.toFixed(2)

  # center_map = [region.centroid().getInfo()['coordinates'][1],region.centroid().getInfo()['coordinates'][0]]
  # Create a folium map object.
  myMap = folium.Map(location=center_map, zoom_start=13, height=500)

  # // Add layers to display
  grayscale = ['f7f7f7', 'cccccc', '969696', '525252'];

  myMap.add_ee_layer(image, {'min': raw10, 'max': raw90, 'palette': grayscale}, 'version')

  
  myMap.add_ee_layer(quartiles, {'min': 1, 'max': 4, 'palette': grayscale}, 'Quartiles')


  myMap.add_ee_layer(zScore, {'min': zScore10, 'max': zScore90, 'palette': grayscale}, 'Z-score')

 
  myMap.add_ee_layer(zScoreMod, {'min': zScoreMod10, 'max': zScoreMod90, 'palette': grayscale}, 'Modified Z-score')

  myMap.add_ee_layer(gearys, {'min': gearys10, 'max': gearys90, 'palette': grayscale}, "Geary's C")

 
  myMap.add_ee_layer(tukeys, {'min': 1, 'max': 2, 'palette': ['22a6ff','ffd400']}, "Tukey's outliers")

 
  myMap.add_ee_layer(zScoreModOutliers, {'min': 3, 'max': 3, 'palette': '13e864'}, 'Modified Z-score outlier')

  myMap.add_ee_layer(gearysOutlier, {'min': 1, 'max': 1, 'palette': ['bebebe']}, "Geary's C design layer")
  myMap.add_ee_layer(gearysOutlier, {'min': 1, 'max': 1, 'palette': ['bebebe']}, "Geary's C outlier")
  myMap.add_ee_layer(combinedMask, {'min': 1, 'max': 5, 'palette': ['6713e8','ff225a','22a6ff','ffd400','13e864']}, 'Combined outliers')

  # Map.addLayer(lteZero, {min: 0, max: 0, palette: ['202020']}, 'Pixel ≤ zero', false);
  myMap.add_ee_layer(lteZero, {'min': 0, 'max': 0, 'palette': ['202020']}, 'Pixel lte zero')

  # empty = ee.Image().byte();
  # var regionVis = empty.paint({
  #   featureCollection: region,
  #   width: 3
  # });
  # Map.addLayer(regionVis, {palette: '252525'}, regionName);
  # Map.centerObject(region);
  myMap.add_child(folium.LayerControl())

  # Display the map.
  display(myMap)

detectOutlires(before_image.select('b1'),'b1',scale=30)
  


In [ ]:
detectOutlires(before_image.select('b4'),'b4')


In [ ]:
# Set visualization parameters.
visParams = {"bands":["b3", "b2", "b1"], "min": 366, "max": 2617,"gamma":2}

# Get AOI centroid
center_map =  aoi.geometry().centroid().getInfo()['coordinates'].reverse() #[aoi.centroid().getInfo()['coordinates'][1],aoi.centroid().getInfo()['coordinates'][0]]

# Create a folium map object.
myMap = folium.Map(location=center_map, zoom_start=13, height=500)

# Add the elevation model to the map object.
planet_scope_mosaic = planet_scope.median()
mapID = ee.Image(planet_scope_mosaic).getMapId(visParams)
myMap.add_ee_layer(planet_scope_mosaic, visParams, 'PlanetScope Mosaic')


myMap.add_ee_layer(before_image, visParams, 'April')
myMap.add_ee_layer(after_image, visParams, 'May')
analog_nir = before_image.select('b4').addBands(after_image.select('b4'))
analog_red = before_image.select('b3').addBands(after_image.select('b3'))
analog_blue = before_image.select('b2').addBands(after_image.select('b2'))
analog_green = before_image.select('b1').addBands(after_image.select('b1'))
analog_ndvi = createNDVI(before_image).select('ndvi').addBands(createNDVI(after_image).select('ndvi'))
analog_bsi = createBSI(before_image).select('bsi').addBands(createBSI(after_image).select('bsi'))


nir_min_max = analog_nir.reduceRegion(reducer=ee.Reducer.minMax(),geometry=aoi,scale=3,maxPixels= 1e12)
print(nir_min_max.getInfo())
nir_min = ee.Number(nir_min_max.get("b4_1_min"))
nir_max = ee.Number(nir_min_max.get("b4_1_max"))

myMap.add_ee_layer(analog_nir, {"bands": ["b4","b4_1","b4_1"], "min":nir_min, "max":nir_max}, 'Analog NIR')

analog_red_min_max = analog_red.reduceRegion(reducer=ee.Reducer.minMax(),geometry=aoi,scale=3,maxPixels= 1e12)
print(analog_red_min_max.getInfo())
analog_red_min = ee.Number(analog_red_min_max.get("b3_1_min"))
analog_red_max = ee.Number(analog_red_min_max.get("b3_1_max"))

myMap.add_ee_layer(analog_red, {"bands": ["b3", "b3_1", "b3_1"], "min":analog_red_min, "max":analog_red_max}, 'Analog B3 Red')

analog_blue_min_max = analog_blue.reduceRegion(reducer=ee.Reducer.minMax(),geometry=aoi,scale=3,maxPixels= 1e12)
print(analog_blue_min_max.getInfo())
analog_blue_min = ee.Number(analog_blue_min_max.get("b2_1_min"))
analog_blue_max = ee.Number(analog_blue_min_max.get("b2_1_max"))
myMap.add_ee_layer(analog_blue, {"bands": ["b2", "b2_1", "b2_1"], "min":analog_blue_min, "max":analog_blue_max}, 'Analog B2 Blue')


myMap.add_ee_layer(analog_green, {"bands": ["b1", "b1_1", "b1_1"], "min":326, "max":4233}, 'Analog B1 Green')


analog_ndvi_min_max = analog_ndvi.reduceRegion(reducer=ee.Reducer.minMax(),geometry=aoi,scale=3,maxPixels= 1e12)
print(analog_ndvi_min_max.getInfo())
analog_ndvi_min = ee.Number(analog_ndvi_min_max.get("ndvi_1_min"))
analog_ndvi_max = ee.Number(analog_ndvi_min_max.get("ndvi_1_max"))
myMap.add_ee_layer(analog_ndvi, {"bands": ["ndvi", "ndvi_1", "ndvi_1"], "min":analog_ndvi_min, "max":analog_ndvi_max}, 'Analog NDVI')

analog_bsi_min_max = analog_bsi.reduceRegion(reducer=ee.Reducer.minMax(),geometry=aoi,scale=3,maxPixels= 1e12)
print(analog_bsi_min_max.getInfo())
analog_bsi_min = ee.Number(analog_bsi_min_max.get("bsi_1_min"))
analog_bsi_max = ee.Number(analog_bsi_min_max.get("bsi_1_max"))
myMap.add_ee_layer(analog_bsi, {"bands": ["bsi", "bsi_1", "bsi_1"], "min":analog_bsi_min, "max":analog_bsi_max}, 'Analog BSI')

# Add study area markers
# folium.GeoJson(cary_park.toGeoJSON()).add_to(myMap)
# folium.GeoJson(centenial.toGeoJSON()).add_to(myMap)

# Add a layer control panel to the map.
myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)

## Generate Change Mask

In [ ]:
water_mask = ps_median.where(classes.eq(5), -9999).neq(-9999).select('b3').selfMask()


binary_red_change = before_image.select('b3').subtract(after_image.select('b3'))#.updateMask(water_mask)
binary_reducer = ee.Reducer.mean().combine(reducer2 = ee.Reducer.stdDev(), sharedInputs=True)

binary_red_std = binary_red_change.reduceRegion(reducer=binary_reducer, geometry=aoi.geometry(),scale=30,maxPixels= 1e12)

neg_3std = binary_red_std.get('b3_mean').getInfo() + (binary_red_std.get('b3_stdDev').getInfo() * -2.5)

red_change_mask = binary_red_change.focal_min(1).lte(neg_3std).selfMask()

sample_mask = binary_red_change.focal_min(1).gt(neg_3std).selfMask()

myMap = folium.Map(location=center_map, zoom_start=15, height=900)

for layer in ps_collection_obcd:
  myMap.add_ee_layer(ps_collection_obcd[layer], {"bands":["b3", "b2", "b1"], "min": 366, "max": 2617,"gamma":2}, "PS %s" % layer)

myMap.add_ee_layer(red_change_mask, {"palette": ['red'], 'opacity': 0.8}, 'Red Change Mask')

myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)

In [ ]:
quantileIndexes(binary_red_change, 'b3')

In [ ]:
detectOutlires(binary_red_change, 'b3', scale=30)

In [ ]:
detectOutlires(binary_nir_change, 'b4')

In [ ]:
objectId = red_change_mask.connectedComponents(ee.Kernel.plus(1), 128)
# objectId128 = red_change_mask.connectedComponents(ee.Kernel.plus(1), 128)
# Compute the number of pixels in each object defined by the "labels" band.
objectSize = objectId.select('labels').connectedPixelCount(128,False)
# Get a pixel area image.
pixelArea = ee.Image.pixelArea()
# Multiply pixel area by the number of pixels in an object to calculate
# the object area. The result is an image where each pixel
# of an object relates the area of the object in m^2.
objectArea = objectSize.multiply(pixelArea)
# Map.addLayer(objectArea, null, 'objectArea');

## Threshold the `objectArea` image to define a mask that will mask out
areaMask = objectArea.gte(1000);

# Update the mask of the `objectId` layer defined previously using the
# minimum area mask just defined.
change_locations = objectId.updateMask(areaMask)

myMap = folium.Map(location=center_map, zoom_start=15, height=900)

for layer in ps_collection_obcd:
  myMap.add_ee_layer(ps_collection_obcd[layer], {"bands":["b3", "b2", "b1"], "min": 366, "max": 2617,"gamma":2}, "PS %s" % layer)

# myMap.add_ee_layer(objectId128.randomVisualizer(), {'opacity': 0.75}, 'objectId128')

myMap.add_ee_layer(change_locations.randomVisualizer(), {'opacity': 0.75}, 'Change Locations')


myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)

Remove Samples from change pixels 

In [ ]:

#Switch Back to OSM classes because the bearren class was acting up
classes_w_barren_change_mask = classes_w_barren.updateMask(sample_mask)
# Get AOI centroid

# Create a folium map object.
myMap = folium.Map(location=center_map, zoom_start=14, height=500)

# Add the elevation model to the map object.
planet_scope_mosaic = planet_scope.median()

planet_scope_mosaic_log = planet_scope.median().log()
planet_scope_mosaic_vis_params = {"bands": ['b3','b2','b1'], 'min':5.67, 'max': 8.39}
myMap.add_ee_layer(planet_scope_mosaic_log, planet_scope_mosaic_vis_params, 'PlanetScope Log Median')


myMap.add_ee_layer(change_locations.randomVisualizer().updateMask(red_change_mask), {'opacity': 0.75}, 'Change Locations')


# Add a layer control panel to the map.
myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)



## Explore Segmentation Cluster Params



In [ ]:
myMap = folium.Map(location=center_map, zoom_start=16, height=900)


myMap.add_ee_layer(ps_median, {"bands":["b3", "b2", "b1"], "min": 366, "max": 2617,"gamma":2}, "PS %s" % layer)

cluster_sizes = [10, 15, 20, 25, 30,35,40,50,100]

for size in cluster_sizes:
    print("Size: %s, Neighborhood: %s" % (size, size * 2))
    seeds = ee.Algorithms.Image.Segmentation.seedGrid(size); #8 was good # 5
    compactness = [0.8]
    for c in compactness:
      # Run SNIC on the regular square grid.
      snic = ee.Algorithms.Image.Segmentation.SNIC(
          size=size,
          image= ps_median, 
          compactness= c, # was 5
          connectivity= 8, # was 8
          neighborhoodSize=size*4, #16was 3,
          seeds= seeds
        )

      snic = snic.select(['b1_mean', 'b2_mean', 'b3_mean', 'b4_mean', 'clusters'], ['b1', 'b2', 'b3', 'b4', 'clusters'])

      clusters = snic.select('clusters')
      myMap.add_ee_layer(clusters.randomVisualizer(), None, 'Cluster %s, Compactness %s' % (size,c))
# train = ps_median.sample(aoi)
# test_seg_classifier = ee.Clusterer.wekaCascadeKMeans().train(train,inputProperties=ps_median.bandNames().getInfo())
# unsup_seg = ps_median.cluster(test_seg_classifier)
# myMap.add_ee_layer(unsup_seg.randomVisualizer(), None, 'Unsupervised Image Seg')

# buildings = classes.select('landclass').eq(1).selfMask()
# myMap.add_ee_layer(buildings, {"palette":['grey']}, 'Buildings')



myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)



## Building and Canopy Extraction

In [ ]:
dem = ee.Image("USGS/NED").select('elevation')
dsm = ee.Image("JAXA/ALOS/AW3D30/V2_2").select('AVE_DSM')
buildings_canopy = dsm.subtract(dem)#.addBands(clusters).reduceConnectedComponents(ee.Reducer.median(), 'clusters',256).rename('object_height')

# Create a folium map object.
myMap = folium.Map(location=center_map, zoom_start=12, height=500)

# Add the elevation model to the map object.
planet_scope_mosaic = planet_scope.median()

planet_scope_mosaic_log = planet_scope.median().log()
planet_scope_mosaic_vis_params = {"bands": ['b3','b2','b1'], 'min':5.67, 'max': 8.39}
myMap.add_ee_layer(planet_scope_mosaic_log.updateMask(aoi_mask), planet_scope_mosaic_vis_params, 'PlanetScope Log Median')

myMap.add_ee_layer(buildings_canopy.updateMask(aoi_mask), {"min": 1, "palette": ['red','orange','yellow', 'purple','lime', 'green'], "opacity": 0.7}, "Building and Canopy")
myMap.add_ee_layer( ps_collection_obcd["before"], planet_scope_mosaic_vis_params, 'PlanetScope Before')


# Add a layer control panel to the map.
myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)

## Classify Data

In [ ]:
# https://gis.stackexchange.com/questions/273658/performing-object-based-image-classification-in-google-earth-engine
ps_collection_obcd_classified = {}
ps_collection_obcd_metrics = {}
 
def classifyImage(image, aoi, scale=3):
# for image in ps_collection_obcd:
  print("Lable: " + image)
  image_label = image
  geometry = aoi
  scale = scale 

  bands = ['b1', 'b2', 'b3', 'b4']

  # Covert 16-bit unsigned integer to 8-bit unsigned
  img = ps_collection_obcd[image].select(bands)
  size = 30
  #seeds = ee.Algorithms.Image.Segmentation.seedGrid(size); #8 was good # 5

  # Run SNIC on the regular square grid.
  #snic = ee.Algorithms.Image.Segmentation.SNIC(
  #  image= img, 
  #  compactness= 0.8, # was 5
   # connectivity= 8, # was 8
  #  neighborhoodSize=int(size*5), #16was 3,
  #  seeds= seeds
  #)

  #snic = snic.select(['b1_mean', 'b2_mean', 'b3_mean', 'b4_mean', 'clusters'], ['b1', 'b2', 'b3', 'b4', 'clusters'])

  #clusters = snic.select('clusters')


  # Compute per-cluster stdDev.
  #stdDev = img.addBands(clusters).reduceConnectedComponents(ee.Reducer.stdDev(), 'clusters', 256)

  
  ndviMedian = createNDVI(img) #.addBands(clusters).reduceConnectedComponents(ee.Reducer.median(), 'clusters',256)
  ndwiMedian = createNDWI(img)#.addBands(clusters).reduceConnectedComponents(ee.Reducer.median(), 'clusters',256)
  bsiMedian = createBSI(img)#.addBands(clusters).reduceConnectedComponents(ee.Reducer.median(), 'clusters',256)
  # dsbiMedian = createDSBI(img).addBands(clusters).reduceConnectedComponents(ee.Reducer.median(), 'clusters',256)

  # Low pass filter
  medianPixel = img.reduceNeighborhood(reducer=ee.Reducer.median(), kernel=ee.Kernel.square(5))
  stdPixel = img.reduceNeighborhood(reducer=ee.Reducer.stdDev(), kernel=ee.Kernel.square(5))
  minMax = img.reduceNeighborhood(reducer=ee.Reducer.minMax(), kernel=ee.Kernel.square(5))
  # canny = ee.Algorithms.CannyEdgeDetector(image=img, threshold= 10, sigma= 1)
  # hough = ee.Algorithms.HoughTransform(canny, 256, 600, 100)
  # ndbsivMedian = createBSI_NDVI_index(bsiMedian.select('bsi'),ndviMedian.select('ndvi')).addBands(clusters).reduceConnectedComponents(ee.Reducer.median(), 'clusters',256)
  # laplacian8 = ee.Kernel.laplacian8(magnitude, normalize)
  glcm_nir = img.select('b3')
  # GLCM only works on Images 32bit and lower
  glcm = glcm_nir.toInt32().glcmTexture(size=3)#.addBands(clusters).reduceConnectedComponents(ee.Reducer.median(), 'clusters',256)



  #Area, Perimeter, Width and Height
  #area = ee.Image.pixelArea().addBands(clusters).reduceConnectedComponents(ee.Reducer.sum(), 'clusters', 256)
  
  
  #minMax = clusters.reduceNeighborhood(ee.Reducer.minMax(), ee.Kernel.square(1));
  
  #perimeterPixels = minMax.select(0).neq(minMax.select(1)).rename('perimeter');

  
  #perimeter = perimeterPixels.addBands(clusters).reduceConnectedComponents(ee.Reducer.sum(), 'clusters', 256);
  

  #sizes = ee.Image.pixelLonLat().addBands(clusters).reduceConnectedComponents(ee.Reducer.minMax(), 'clusters', 256)
  #width = sizes.select('longitude_max').subtract(sizes.select('longitude_min')).rename('width')
  #height = sizes.select('latitude_max').subtract(sizes.select('latitude_min')).rename('height')

  #gives metrics of shape smoothness (low) or roughtness (high)
  #shape_index = perimeter.divide(area.sqrt().multiply(4)) #Chapter 9 p.419 IDIP
  
  objectPropertiesImage = ee.Image.cat([
    img.select(['b1', 'b4']),
    medianPixel.select(['b1_median', 'b4_median']),
    stdPixel.select(['b1_stdDev','b4_stdDev']),
    # hough,
    minMax.select(['b1_min','b1_max', 'b4_min', 'b4_max']),
    # stdDev.select(['b3', 'b4']),
    ndviMedian.select('ndvi'),
    ndwiMedian.select('ndwi'),
    # dsbiMedian.select('dsbi'),
    bsiMedian.select('bsi'),
    # ndbsivMedian.select('ndbsiv'),
    glcm.select('b3_contrast', 'b3_asm', 'b3_corr'),
    # area,
    # perimeter,
    # width,
    # height,
    # shape_index
    # buildings_canopy.clipToCollection(aoi)
  ]).float();
  print("objects properties image created")

  #Mean center the data to enable a faster covariance reducer
  #and an SD stretch of the principal components.
  bandNames = objectPropertiesImage.bandNames().getInfo()
  meanDict = objectPropertiesImage.reduceRegion(reducer= ee.Reducer.mean(), geometry = geometry, scale=scale, maxPixels=1e12, bestEffort=True, tileScale=16)
  means = ee.Image.constant(meanDict.values(bandNames))
  centered = objectPropertiesImage.subtract(means);
  print("PCA:")
  # pcImage = getPrincipalComponents(objectPropertiesImage, scale, geometry).float()
  pcImage = objectPropertiesImage
 

  trainingPartition = pcImage.addBands(classes_w_barren_change_mask.select('landclass')).stratifiedSample(numPoints=20000,
      classBand='landclass',
      classValues=[0,1,2,3,4,5,6],
      # classPoints=[1000,1000,1000,1000,1000,1000,1000], #[0,2000,2500,5000,5000,0,2000],
      classPoints=[0,7540, 3530, 1400, 3750, 0, 2580],
      scale=scale,
      # tileScale=4,
      seed=10,
      region=aoi)
  
  # trainingPartition = pcImage.sampleRegions(
  #     collection= combined_feature_collection,
  #     properties=['landclass'],
  #     scale=scale)
  

  print("training samples genereated")
  withRandom = trainingPartition.randomColumn();

  split = 0.7  # Roughly 70% training, 30% testing.
  trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
  testingPartition = withRandom.filter(ee.Filter.gte('random',split))

  #Train from all sample data and test from clusters
  # if image == "after":
  #   pcImage = pcImage.updateMask(change_locations.select('labels'))

  # testingPartition = pcImage.addBands(classes_w_barren_change_mask.select('landclass')).stratifiedSample(numPoints=10000,
  #     classBand='landclass',
  #     classValues=[0,1,2,3,4,5,6],
  #     classPoints=stratifed_samples_sizes,
  #     scale=scale,
  #     tileScale=4,
  #     seed=40,
  #     region=aoi)

  white_list_features = pcImage.bandNames().getInfo()
  print(white_list_features)

  def autoMLRF(image, training, testing):

    white_list_features = image.bandNames().getInfo()
    num_trees = [10,50,100,250,500] #,1000]
    winner_score = 0
    winner = None
    winner_trees = 0
    for t in num_trees:
      classifier = ee.Classifier.smileRandomForest(numberOfTrees=t).train(features=training, classProperty='landclass',inputProperties= white_list_features)
      validated = testing.classify(classifier)
      testAccuracy = validated.errorMatrix('landclass', 'classification')
      overallAccuracy = testAccuracy.accuracy().getInfo()
      print("Trees: %s, OA: %s" %(t, overallAccuracy))
      if winner_score < overallAccuracy:
        winner_score = overallAccuracy
        winner = classifier
        winner_trees = t
  
    print("Winner had Trees: %s, OA: %s" %(winner_trees, winner_score))
    # Get the most important features
    white_list_feature_importance = createFeatureImportanceBarChart(winner, image_label).tolist()
    print(white_list_feature_importance)
    # Rerun winning classifier with important features
    # winner = ee.Classifier.smileRandomForest(winner_trees).train(features=training, classProperty='landclass',inputProperties= white_list_features)
    return winner

  classifier = autoMLRF(pcImage, trainingPartition,testingPartition)
    
  #Classify the validation data.
  validated = testingPartition.classify(classifier)

  #post-processing
  # water = classes_w_barren.select('landclass').eq(5).selfMask()
  # roads = road_class.select('landclass').eq(0).selfMask().clipToCollection(aoi)
  # notWater = classes_w_barren.select('landclass').neq(5).selfMask()
  # validated = validated.where(water, water).where(roads, roads)

  #Get a confusion matrix representing expected accuracy.
  testAccuracy = validated.errorMatrix('landclass', 'classification')
  print(' Resubstitution error matrix: ', testAccuracy.getInfo())
  print(' Training overall accuracy: ', testAccuracy.accuracy().getInfo())
  print(' Consumers accuracy: ', testAccuracy.consumersAccuracy().getInfo())
  print(' Producers accuracy: ', testAccuracy.producersAccuracy().getInfo())
  print(' Kappa: ', testAccuracy.kappa().getInfo())
  # createConfusionMatixFigure(testAccuracy, label=image)


  print("Classifier Created")
  #Classify the image with the same bands used for training.
  #Mask out water since we are adding that in post processing
  classified = pcImage.select(white_list_features).classify(classifier)
  
  #Apply median low-pass filter to smooth results
  # post_classification = post_classification.focal_mode(1,'square', 'meters')
  #Add water and roads from OSM to classification output
  post_classification = (
      classified
        # .where(classes_w_barren.select('landclass').eq(5), water.multiply(5))
        .where(water_class, 5)
        .where(road_class.select('landclass').eq(0),0)
        )
  
    
  print("Image Classified")
  ps_collection_obcd_metrics[image] = {}
  ps_collection_obcd_metrics[image]["training"] = trainingPartition
  ps_collection_obcd_metrics[image]["testing"] = testingPartition
  ps_collection_obcd_metrics[image]["rawClassification"] = classified
  ps_collection_obcd_metrics[image]["classifier"] = classifier
  ps_collection_obcd_metrics[image]["accuracy"] = testAccuracy

  ps_collection_obcd_classified[image] = post_classification
  return classified

In [ ]:
# for image in ps_collection_obcd:
before_classified = classifyImage('before', aoi_bbox,scale=3)
# exportEarthEngineImage(
    # image=classifyImage('before', aoi_bbox),
    # desc="before_image_classified",
    # imageName="before_image_classified",
    # region=aoi_bbox,
    # saveLocation="CloudStorage")

In [ ]:
createConfusionMatixFigure(ps_collection_obcd_metrics["before"]["accuracy"], label="before")

In [ ]:
after_classified = classifyImage('after', aoi_bbox, scale=3)

In [ ]:
print(ps_collection_obcd_metrics)
createConfusionMatixFigure(ps_collection_obcd_metrics["after"]["accuracy"], label="after")

In [ ]:
test = ps_collection_obcd_metrics["after"]["training"].toList(10000).getInfo()
type(test)
list_of_properties = [f['properties'] for f in test]
df = pd.DataFrame(list_of_properties)

df["landcover"] = df['landclass'].apply(str).map({
    "0": 'road',
    "1": 'building',
    "2": 'barren',
    "3": 'forest',
    "4": 'grass',
    "5": 'water',
    "6": 'developed'
})

df.head(5)

In [ ]:
df.groupby("landcover").describe()

In [ ]:

pair_plot_palette = landcover_color_palette[1:5] + landcover_color_palette[6:7]

sns.set_palette(sns.color_palette(["#" + c for c in pair_plot_palette]))
sns.pairplot(df[['ndvi', 'ndwi', 'b1', 'b4', 'bsi','b4_median','b4_min', 'landcover']], hue='landcover')
# ['pc3', 'pc7', 'pc10', 'pc13', 'pc11']

# sns.pairplot(df[['pc12', 'pc4', 'pc3', 'pc11', 'pc6', 'landcover']], hue="landcover")

In [ ]:
# water = classes.select('landclass').eq(5).selfMask()
# barren = classes.select('landclass').eq(2).selfMask()
  # classified = classified.cat(water)
center_map = [cary_park.centroid().getInfo()['coordinates'][1],cary_park.centroid().getInfo()['coordinates'][0]]
myMap = folium.Map(location=center_map, zoom_start=15, height=900)

# myMap.add_ee_layer(water, {"palette": ['blue']}, 'water')


for layer in ps_collection_obcd_classified:
  myMap.add_ee_layer(ps_collection_obcd[layer].updateMask(aoi_mask), {"bands":["b3", "b2", "b1"], "min": 366, "max": 2617,"gamma":2}, "PS %s" % layer)

# for from_image_key in ps_collection_obcd_classified:
myMap.add_ee_layer(ps_collection_obcd_classified["before"].updateMask(aoi_mask), {"min": 0, "max": 6, "palette": landcover_color_palette, "opacity": 0.6}, "%s" % "before")
myMap.add_ee_layer(ps_collection_obcd_classified["after"].updateMask(aoi_mask), {"min": 0, "max": 6, "palette": landcover_color_palette, "opacity": 0.6}, "%s" % "after")



# nlcd = ee.Image('USGS/NLCD/NLCD2016').clip(aoi)
# myMap.add_ee_layer(nlcd.select('landcover'), {}, 'NLCD')
# myMap.add_ee_layer(change_locations.randomVisualizer(), {}, "Binary Change Mask")

# myMap.add_ee_layer(barren, {"palette": ['olive']}, 'barren')
myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)


Export Classified Maps

In [ ]:
# exportToDrive(ps_collection_obcd['before'],"classified_before_30m", resolution=30)
# exportToDrive(ps_collection_obcd['after'],"classified_after_30m", resolution=30)
print(f"Before Classified: {ps_collection_obcd_classified['before'].bandNames().getInfo()}")
print(f"After Classified: {ps_collection_obcd_classified['after'].bandNames().getInfo()}")

ee.batch.Export.image.toDrive(
    image=ps_collection_obcd_classified['before'],
    folder=figures_save_location,
    description='classified_before_3m',
    fileNamePrefix='classified_before_3m',
    scale=3,
    fileFormat= 'GeoTIFF',
    region=aoi_bbox.geometry(),
    formatOptions= {"cloudOptimized": True}).start()


ee.batch.Export.image.toDrive(
    image=ps_collection_obcd_classified['after'],
    folder=figures_save_location,
    description='classified_after_3m',
    fileNamePrefix='classified_after_3m',
    scale=3,
    fileFormat= 'GeoTIFF',
    region=aoi_bbox.geometry(),
    formatOptions= {"cloudOptimized": True}).start()


## Generate Thematic Change Maps

In [ ]:
ps_thematic_change_collection_obcd = {}
for from_image_key in ps_collection_obcd_classified:
  from_image = ps_collection_obcd_classified[from_image_key]
  for to_image_key in ps_collection_obcd_classified:
    if from_image_key != to_image_key:
      to_image = ps_collection_obcd_classified[to_image_key]
      thematic_change_key = "from_%s_to_%s" % (from_image_key, to_image_key)
      thematic_change_image = generateThematicChangeImage(from_image, to_image, thematic_change_expression)
      ps_thematic_change_collection_obcd[thematic_change_key] = thematic_change_image

In [ ]:
expression = ""
for index, row in df_change_classes.iterrows():
  expression = expression + "(b('constant') == {}) ? {} :".format(row['ClassID'],row["priority"])

expression = expression + " 0"

print(expression)

In [ ]:

print(change_locations.bandNames().getInfo())

def getImageTotalArea(image):
  print("AreaImage Start")
  areaImage = image.multiply(0).rename('area')
  print("AreaImage End")
  totalArea = ee.Number(areaImage.add(1).reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= aoi,
        scale= 30,
        maxPixels= 1e12).get('area'))
  return totalArea.divide(1e6).getInfo()

print("totalArea: {} km2".format(getImageTotalArea(ps_median.select('b3'))))
print("Total Change Area: {} km2".format(getImageTotalArea(change_locations.select('b3'))))

In [ ]:
priority_change = (ps_thematic_change_collection_obcd['from_before_to_after']
                   .expression(expression)
                   .select(['constant'],['priority'])) 
                  #.clip(aoi)
                  #.updateMask(change_locations.select('labels'))
print("priority_change")
print(priority_change.bandNames().getInfo())

In [ ]:
exportToDrive(priority_change,"priority_change_30m", resolution=30)


In [ ]:
def printMinMax(image, imageName):
  print("{0}:{1}".format(imageName, image.reduceRegion(reducer= ee.Reducer.minMax(), geometry=aoi, scale=30,maxPixels= 1e16).getInfo()))

priority_change_objectId = priority_change.addBands(change_locations.select("labels"))
objectSize = priority_change_objectId.select('labels').connectedPixelCount(128,False)
print("objectSize: {}".format(objectSize.bandNames().getInfo()))
# printMinMax(objectSize,"objectSize")
# Get a pixel area image.

# Make sure pixels are set to correct scale 3m = 9m2
pixelArea = priority_change_objectId.select('labels').multiply(0).add(9).rename('area')

# pixelArea = ee.Image.pixelArea()
printMinMax(pixelArea,"pixelArea")

# Multiply pixel area by the number of pixels in an object to calculate
# the object area. The result is an image where each pixel
# of an object relates the area of the object in m^2.
objectArea = objectSize.multiply(pixelArea).select(['labels'], ['area'])
print("objectArea: {}".format(objectArea.bandNames().getInfo()))
printMinMax(objectArea,"objectArea")



print("priority_change_objectId: {}".format(priority_change_objectId.bandNames().getInfo()))


#Get the mean of the change areas priority and multiple it by the objects area
objectPriority = priority_change_objectId.reduceConnectedComponents(reducer=ee.Reducer.mean(),labelBand= 'labels')
print("objectPriority: {}".format(objectPriority.bandNames().getInfo()))
# printMinMax(objectPriority,"objectPriority")


reducer = ee.Reducer.mean().splitWeights()
objectPriorityWeighted = priority_change_objectId.addBands(objectArea.select(['area'])).reduceConnectedComponents(reducer=reducer,labelBand= 'labels')
print("objectPriorityWeighted: {}".format(objectPriorityWeighted.bandNames().getInfo()))
# printMinMax(objectPriority,"objectPriority")

# objectPriorityWeighted = objectPriority.divide(objectSize)
# print("objectPriorityWeighted: {}".format(objectPriorityWeighted.bandNames().getInfo()))
# printMinMax(objectPriorityWeighted,"objectPriorityWeighted")


priorityQueue = objectPriority.addBands([priority_change_objectId.select('labels'),objectArea.select('area')]).reduceConnectedComponents(reducer=ee.Reducer.product(),labelBand= 'labels')
print("priorityQueue: {}".format(priorityQueue.bandNames().getInfo()))
# printMinMax(priorityQueue,"priorityQueue")


priorityQueueMutli = objectPriority.multiply(objectArea)
print("priorityQueueMutli: {}".format(priorityQueueMutli.bandNames().getInfo()))
# printMinMax(priorityQueueMutli,"priorityQueueMutli")



myMap = folium.Map(location=center_map, zoom_start=16, height=900)

for layer in ps_collection_obcd:
  myMap.add_ee_layer(ps_collection_obcd[layer], {"bands":["b3", "b2", "b1"], "min": 366, "max": 2617,"gamma":2}, "PS %s" % layer)

# Priority Map
priority_change_viz = {"min": 0, "max": 7, 'palette': ['feebe2','fcc5c0','fa9fb5','f768a1','dd3497','ae017e','7a0177']}
myMap.add_ee_layer(priority_change, priority_change_viz, "Priority Change")


# Mean Object Priority 
priority_queue_viz = {'opacity': 1, 'palette': ["ffffb2","fecc5c","fd8d3c","f03b20","FF0000"]}
myMap.add_ee_layer(objectPriority.select('priority'), priority_queue_viz, 'Object Priority')
myMap.add_ee_layer(objectPriorityWeighted.select('mean'), priority_queue_viz, 'Object Priority Weighted')

# Obejct Size eg number of pixels
object_size_viz = {'min':52, 'max':128,'opacity': 1, 'palette': ["ffffb2","fecc5c","fd8d3c","f03b20","FF0000"]}
myMap.add_ee_layer(objectSize, object_size_viz, 'Object Size')

# Object Area
object_area_viz = {'min':468, 'max':1152,'opacity': 1, 'palette': ["ffffb2","fecc5c","fd8d3c","f03b20","FF0000"]}
myMap.add_ee_layer(objectArea, object_area_viz, 'Object Area')


priority_queue_viz = {'opacity': 1, 'palette': ["ffffb2","fecc5c","fd8d3c","f03b20","FF0000"]}
myMap.add_ee_layer(priorityQueue.select('priority'), priority_queue_viz, 'Priority Queue')
myMap.add_ee_layer(priorityQueueMutli.select('priority'), priority_queue_viz, 'Priority Queue Multi')






myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)

In [ ]:
print(priority_change.bandNames().getInfo())
print(priorityQueueMutli.bandNames().getInfo())


In [ ]:
printMinMax(priorityQueueMutli,"priorityQueueMutli")


In [ ]:
print(priorityQueueMutli.getInfo())

In [ ]:
ee.batch.Export.image.toDrive(
    image=priorityQueue,
    folder=figures_save_location,
    description='priority_queue_30m',
    fileNamePrefix='priority_queue_30m',
    # fileNamePrefix='gee_priority_queue_multi_30m',
    scale=30,
    fileFormat= 'GeoTIFF',
    region=aoi.geometry(),
    # region=cary_park,
    formatOptions= {"cloudOptimized": True}).start()

ee.batch.Export.image.toDrive(
    image=priority_change,
    folder=figures_save_location,
    description='priority_change_30m',
    fileNamePrefix='priority_change_30m',
    # fileNamePrefix='gee_priority_queue_multi_30m',
    scale=30,
    fileFormat= 'GeoTIFF',
    region=aoi.geometry(),
    # region=cary_park,
    formatOptions= {"cloudOptimized": True}).start()

In [ ]:
priority_queue_viz = {
    'palette': ["ffffb2","fecc5c","fd8d3c","f03b20","FF0000"]
    }

ee.batch.Export.image.toDrive(
    # image=priority_change.select('priority'),
    # image=priorityQueue,
    # image=priorityQueue.addBands(change_locations.select("labels")).toDouble(),
    image=priorityQueueMutli, #.addBands(change_locations.select("labels")).toDouble(),
    folder=figures_save_location,
    description='priorit30myQueueMutli30m',
    fileNamePrefix='priorityQueueMutli30m',
    # fileNamePrefix='gee_priority_queue_multi_30m',
    scale=30,
    fileFormat= 'GeoTIFF',
    region=aoi.geometry(),
    # region=cary_park,
    formatOptions= {"cloudOptimized": True}).start()

In [ ]:
priorityQueueMutli.getDownloadURL({
    "palette":["ffffb2","fecc5c","fd8d3c","f03b20","FF0000"], 
    "bands":["priority"],
    "scale": 30,
    "region":aoi.geometry()})

In [ ]:
# priority_change_connected_components = priority_change.connectedComponents(ee.Kernal.square(10, 'pixels', True))
#Uniquely label the hotspot image objects.
hotspots = priority_change.updateMask(priority_change.gt(5))

objectId = hotspots.connectedComponents(connectedness = ee.Kernel.square(1),maxSize = 128)
#Compute the number of pixels in each object defined by the "labels" band.
objectSize = objectId.select('labels').connectedPixelCount(maxSize= 128, eightConnected= True)
#Get a pixel area image.
pixelArea = ee.Image.pixelArea()
#Multiply pixel area by the number of pixels in an object to calculate
#the object area. The result is an image where each pixel
#of an object relates the area of the object in m^2.
objectArea = objectSize.multiply(pixelArea)
areaMask = objectArea.gte(400)
# Make a suitable image for `reduceConnectedComponents()` by adding a label
# band to the `kelvin` temperature image.
hotspots = hotspots.addBands(objectId.select(['labels']))
#Calculate the mean temperature per object defined by the previously added
#"labels" band.
patchTemp = hotspots.reduceConnectedComponents(reducer = ee.Reducer.mean(),labelBand = 'labels',maxSize = 128)
# print(patchTemp.bandNames().getInfo())
#weight x area 
patchTempWeighted = patchTemp.multiply(objectArea).updateMask(areaMask)

filter_geom = centenial
# Normalize values between [0, 1]
reducedDict = patchTempWeighted.reduceRegion(ee.Reducer.minMax(), filter_geom, scale=3, bestEffort= True, tileScale=16, crs='EPSG:3857',maxPixels= 1e12)

# stdDev = ee.Number(image.reduceRegion(
#       reducer= ee.Reducer.stdDev(),
#       geometry= region,
#       scale= scale,
#       maxPixels= 1e12).get(band));

# print(reducedDict.getInfo())
minValue = reducedDict.get("constant_min").getInfo()
maxValue = reducedDict.get("constant_max").getInfo()
print(minValue)
print(maxValue)
normalizedPriorityChange = patchTempWeighted.unitScale(minValue, maxValue)




In [ ]:
# Get AOI centroid
center_map = [filter_geom.centroid().getInfo()['coordinates'][1],filter_geom.centroid().getInfo()['coordinates'][0]]

# Create a folium map object.
myMap = folium.Map(location=center_map, zoom_start=16, height=900)

for layer in ps_collection_obcd_classified:
  myMap.add_ee_layer(ps_collection_obcd[layer], {"bands":["b3", "b2", "b1"], "min": 366, "max": 2617,"gamma":2}, "PS %s" % layer)

# for from_image_key in ps_collection_obcd_classified:
#   myMap.add_ee_layer(ps_collection_obcd_classified[from_image_key], {"min": 0, "max": 5, "palette": landcover_color_palette, "opacity": 0.4}, "%s" % from_image_key)

# thematic_change = ps_thematic_change_collection_obcd['from_before_to_after']


                     
mask = priority_change.updateMask(priority_change.neq(0))
mask = mask.focal_median(3, 'square')
myMap.add_ee_layer(mask.updateMask(mask.gt(3)), {"palette": ['green','yellow','orange','red', 'purple'], "opacity":0.5}, "Priority Change")
high_priority_mask = priority_change.updateMask(priority_change.eq(7))
myMap.add_ee_layer(high_priority_mask, {"palette": ['purple'], "opacity":0.7}, "High Priority Change")


myMap.add_ee_layer(patchTempWeighted, {"palette": ['ffffd9','edf8b1','c7e9b4','7fcdbb','41b6c4','1d91c0','225ea8','253494','081d58'], "opacity":0.7}, 'patchTempWeighted')

myMap.add_ee_layer(normalized_priority_change, {"min": 0, "max": 1, "palette": ['ffffd9','edf8b1','c7e9b4','7fcdbb','41b6c4','1d91c0','225ea8','253494','081d58'], "opacity":0.7}, 'normalizedpatchTempWeighted')
# myMap.add_ee_layer(normalized_priority_change, {"min": 0, "max": 1, "palette": ['orange', 'purple'], "opacity":0.7}, 'patchTempWeighted Orange')

myMap.add_ee_layer(binary_change_mask_red.updateMask(red_change_mask), {"max": 1, 'min': 0, "palette": ['orange', 'purple'], 'opacity': 0.75}, 'Red Binary')

# normalized = normalizedPriorityChange.focal_median(500,'square','meters')
# myMap.add_ee_layer(normalized, {"min": 0, "max": 1, "palette": ['ffffd9','edf8b1','c7e9b4','7fcdbb','41b6c4','1d91c0','225ea8','253494','081d58'], "opacity":0.7}, 'patchTempWeighted normalized image')




# folium.GeoJson(aoi.toGeoJSON(), name="Study Area").add_to(myMap)
# Add a layer control panel to the map.
myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)

In [ ]:
class_vis_Params = {'min':0, 'max':1, 'dimensions': 1000, "palette": ['ffffd9','edf8b1','c7e9b4','7fcdbb','41b6c4','1d91c0','225ea8','253494','081d58']}
Image(url=normalized.clip(aoi)
  .getThumbUrl(class_vis_Params))

## Export Data to Google Cloud Storage

In [ ]:
#Export High Priority Change
exportEarthEngineImage(
    image=normalizedPriorityChange.visualize(bands="constant", min=0, max=1, palette=['yellow','orange', 'red', 'purple']),
    desc="HighPriorityChangeMapCentenial", 
    imageName="HighPriorityChangeCentenial",
    region=centenial, 
    saveLocation="CloudStorage")

In [ ]:
#Export AOI
for layer in ps_collection_obcd:
  exportEarthEngineImage(
    image=ps_collection_obcd[layer],
    desc="AOI %s" % layer, 
    imageName="AOI_%s" % layer,
    region=aoi, 
    saveLocation="CloudStorage")

In [ ]:
#Export LULC Maps
for from_image_key in ps_collection_obcd_classified:
  exportEarthEngineImage(
    image=ps_collection_obcd_classified[from_image_key],
    desc="LULC %s" % layer, 
    imageName="LULC_%s" % layer,
    region=aoi, 
    saveLocation="CloudStorage")

In [ ]:
 exportEarthEngineImage(
    image=thematic_change,
    desc="Thematic Change Map Raw", 
    imageName="Raw_Thematic_Change",
    region=aoi, 
    saveLocation="CloudStorage")